# SAP to Databricks Code Conversion

# How to use:

Put the prompt in the Prompt Cell

Put the logic / calculation view to be converted in the Logic Cell

**Run the Notebook by using the "cluster_deloitte_shared" cluster.**


# Config File maintenance

## Import required modules (libraries)

In [0]:
# Import required modules (libraries)
from datetime import datetime
import os
import time
import pandas as pd
import re
import concurrent.futures


In [0]:
 # This will allow widget creation in the source notebook
is_target_notebook = True 

## Import required functions

In [0]:
%run ./init

In [0]:
# Function to call the appropriate model based on selected type
def call_model(selected_model, prompt):
    if selected_model == "o1":
        return call_model_o1(deployment_name="o1", prompt=prompt)  # Call the o1 model function
    elif selected_model == "o1-preview":
        return callmodelo1(deployment_name="o1-preview", prompt=prompt)  # Call the o1-preview model function
    elif selected_model == "o1-mini":
        return callmodelo1(deployment_name="o1-mini", prompt=prompt)  # Call the o1-mini model function
    else:
        raise ValueError("Invalid model choice")


## Get parameter value(s)

In [0]:

# Create a dropdown widget for selecting model type (o1 or o1 mini)
dbutils.widgets.dropdown("model_choice", "o1", ["o1", "o1-mini", "o1-preview"], "Choose model type")
# Retrieve the selected model type
selected_model = dbutils.widgets.get("model_choice")
print(f"Selected model: {selected_model}")


In [0]:
# Get parameters value
data_flow_man = dbutils.widgets.get("data_flow_insertion")
data_flow_man = data_flow_man.upper()
print(f"Current working directory: {data_flow_man}")

In [0]:
# # Set your storage account SAS token and name
# sasToken = dbutils.secrets.get(scope="codeconv", key="sasTokenKey") # Create a new scope (directly from databricks UI) and secret (in Key-Vault)
# #sa = "erdccalearning" # Replace with your actual storage account name
# sa = dbutils.secrets.get(scope="codeconv", key="storageAccountName")
# container_name = "nndemo" # Replace with your actual container name
# spark.conf.set(f"fs.azure.account.auth.type.{sa}.dfs.core.windows.net", "SAS")
# spark.conf.set(f"fs.azure.sas.token.provider.type.{sa}.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.sas.FixedSASTokenProvider")
# spark.conf.set(f"fs.azure.sas.nndemo.{sa}.blob.core.windows.net", f"{sasToken}")

sa = "stlpdel01dev"
container_name = "codeconverter" # Replace with your actual container name
spark.conf.set("fs.azure.account.auth.type", "CustomAccessToken")
spark.conf.set("fs.azure.account.custom.token.provider.class", spark.conf.get("spark.databricks.passthrough.adls.gen2.tokenProviderClassName"))

## Get prompt

In [0]:
prompt = """

["Convert a complex SAP HANA Calculation View (represented in XML format) into optimized Databricks SQL code. The data sources are already available as Delta tables (representing ADSOs) in Databricks. The task involves parsing multiple input files (in batches) and converting the logic—even if it's complex—into a single optimized SQL query.", '', 'Key Requirements:', '', 'Parse Complex Logic and Multiple Files:', 'Ensure that the model parses all provided batch files, regardless of the complexity. Even if the logic is intricate (involving multiple joins, calculations, and aggregations), ensure that the model processes them accurately.', '', 'Joins Translation:', 'Convert the joins between the tables in SAP HANA to the corresponding Databricks SQL joins. Pay attention to how joins are defined in the Calculation View XML and convert them precisely into SQL syntax.', '', 'Calculations Conversion:', 'Parse all the calculations defined in the Calculation View XML and convert them into Databricks SQL expressions. This includes any field transformations, custom formulas, and aggregations.', '', 'Avoid Duplicate Columns:', 'Ensure that after joining the tables, no duplicate columns exist. Use the <input node=> tags from the HANA XML to determine which column should be retained from each table. This helps in eliminating conflicts or redundancy.', '', 'Optimize the Query for Databricks SQL:', 'Convert the Calculation View’s logic into a Databricks SQL query optimized for performance. This involves ensuring that the query is efficient by following best practices such as minimizing unnecessary computations, ensuring proper indexing, and reducing the number of joins when possible. When optimizing, avoid using CTEs as it introduces difficulties in troubleshooting and code reading.', '', 'Handling Batch Files:', 'If multiple files or batches are provided, parse each one, and then integrate them into a single coherent SQL query. Even if the files are large or contain complex logic, handle them in a manner that the final result is efficient and readable.', '', 'Specific Tasks:', '', 'Joins:', 'Parse the provided batch files and identify all the join conditions. Convert these into the appropriate Databricks SQL join syntax (JOIN, LEFT JOIN, etc.). Take special care to handle any complex or nested joins.', '', 'Calculations:', 'For any calculations (e.g., derived fields, aggregations, or custom formulas), convert them into equivalent Databricks SQL expressions. Ensure these are applied at the appropriate stages in the query (e.g., after joins or within subqueries).', '', 'Column Selection:', 'Make sure that each column is sourced correctly according to the <input node=> mapping from the HANA XML. This ensures the final result does not have duplicate columns and keeps only the necessary fields from each input table. When writing SQL queries, please make sure to explicitly define each column you are selecting. Avoid using SELECT *, and instead, list out the specific columns needed. This will help in optimizing performance, making the query more readable, and ensuring clarity in the data being fetched.', '', "Remove the 'CV_' prefix from the table names, ensuring that both uppercase and lowercase variations of 'CV_' are handled.", '', 'Query Optimization:', '', 'Avoid selecting unnecessary columns.', 'Consider using WITH clauses for intermediate results.', 'Use efficient SQL operations that Databricks SQL supports for large datasets (e.g., MERGE, OPTIMIZE, etc.).', 'Pay attention to query performance, especially for large data batches.', 'Expected Output:', 'Provide the optimized Databricks SQL code for the final query after all batch files are parsed, with detailed inline comments explaining:', 'How each join is performed.', 'The purpose of each calculation.', 'How duplicates are avoided by following <input node=> tags.', 'Any optimizations made to improve performance (e.g., usage of WITH clauses, proper indexing, etc.).', '', '', 'Document joining conditions, filtering, selected columns]. Make sure you don't skip any piece of logic and convert the XML file entirely. When writing SQL queries, make sure to explicitly define each column you are selecting. Avoid using SELECT *, and instead, list out the specific columns needed. This will help in optimizing performance, making the query more readable, and ensuring clarity in the data being fetched.


"""

In [0]:
print(prompt)

## Get Source Code

In [0]:
logic = ("""

<?xml version="1.0" encoding="UTF-8"?>
<View:ColumnView xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:Column="http://www.sap.com/ndb/DataModelFilter.ecore" xmlns:Param="http://www.sap.com/ndb/ViewModelParameter.ecore" xmlns:Type="http://www.sap.com/ndb/DataModelType.ecore" xmlns:View="http://www.sap.com/ndb/ViewModelView.ecore" schemaVersion="2.3" name="CV_DLO_PX_DEPOT" dataCategory="DEFAULT" hierarchiesSQLEnabled="false" defaultNode="#/0/Projection" clientDependent="true" applyPrivilegeType="ANALYTIC_PRIVILEGE" translationRelevant="true">
  <endUserTexts label="CV_DLO_PX_DEPOT">
    <comment text="6.2.3&#x9;Determine Number of years for Depotfinanciering&#xD;&#xA;For depots, a normal Lego RELNR has a relation with Lego Depot RELNR. This relation is not stored anywhere in Lego, but is only know to a number of persons in the L&amp;P department.   &#xD;&#xA;An overview has been created where all know depot relations are available. This overview will be needed as a source for the determination of the number of years for Depotfinanciering. &#xD;&#xA;&#xD;&#xA;The sheet DEPOT_REL contains the following fields: &#xD;&#xA;RELNR &#xD;&#xA;GRPNR&#xD;&#xA;TOESL_CAT_A_I&#xD;&#xA;RELNR_depot&#xD;&#xA;&#xD;&#xA;The sheet will NOT contain a ReferenceDate. Christine Kong and John Oost will maintain the sheet. Once a year in December just before the 31-12 MPO Run our team needs to send the latest version and they will do the updates and send the file back. The file can be loaded with a load datetime as ValidFrom. When selecting the records the selection will be:&#xD;&#xA;1. Determine the most recent load &#xD;&#xA;Select the most recent ValidFrom &#xD;&#xA;2. Select all records with ValidFrom of step 1&#xD;&#xA;This will be the records used in the further selection.&#xD;&#xA;&#xD;&#xA;Records do need to be splitted into 2 groups:&#xD;&#xA;•&#x9;Records having a filled DEPOT_REL.GRPNR&#xD;&#xA;•&#x9;Records having an empty DEPOT_REL.GRPNR&#xD;&#xA;&#xD;&#xA;IF a record has DEPOT_REL.GRPNR filled then the join to find a concerning DEPOT_REL.RELNR_depot must also be on the BASE.GRPNR&#xD;&#xA;ELSE the join must be only on BASE.RELNR. &#xD;&#xA;&#xD;&#xA;For each RELNR in the Base:&#xD;&#xA;    Find the record in DEPOT_REL where&#xD;&#xA;    DEPOT_REL.RELNR = BASE.RELNR (and&#xD;&#xA;    DEPOT_REL.GRPNR = BASE.GRPNR) [brackets part depends on the above IF condition]&#xD;&#xA; &#xD;&#xA;The result is a list of RELNR_depot that are linked to RELNR. There is also a field DEPOT_REL.TOESL_CAT_A_I that explains if the link is for an Active or Inactive toeslag. There are 3 flavors:&#xD;&#xA;-&#x9;A  = link to RELNR_depot is only for an Active toeslag&#xD;&#xA;-&#x9;I   = link to RELNR_depot is only for an Inactive toeslag&#xD;&#xA;-&#x9;AI = link to RELNR_depot is for both an Active as an Inactive toeslag&#xD;&#xA;&#xD;&#xA;On the level of a BASE record it’s stored if a BASE record has an Active and/or Inactive toeslag. And also for Depotfinanciering only certain categories do apply. So the first thing is to ONLY select the applicable BASE records based on ACT_FIN_TYPE and ACT_TOESL_CAT&#xD;&#xA;&#xD;&#xA;To limit the lookup ONLY select the BASE records that do have a BASE.RELNR and in some cases also the BASE.GRPNR must be combined that are present or mentioned in the DEPOT_REL table.&#xD;&#xA;&#xD;&#xA;Then also ONLY select those records where:&#xD;&#xA;&#xD;&#xA;IF (BASE.ACT_FIN_TYPE in (3, 7, 8) and&#xD;&#xA;     BASE.ACT_TOESL_CAT in (3, 4)) &#xD;&#xA; Mark this BASE record as “Active”&#xD;&#xA;&#xD;&#xA;IF (BASE.INACT_FIN_TYPE in (3, 7, 8) and&#xD;&#xA;     BASE.INACT_TOESL_CAT in (3, 4)) and&#xD;&#xA; Mark this BASE record as “Inactive”&#xD;&#xA; &#xD;&#xA;It can be that a BASE record is Active and Inactive. It means that a record can be marked as Active and Inactive at the same time.&#xD;&#xA;&#xD;&#xA;Example of a BASE record that is Active and Inactive at the same time.&#xD;&#xA;&#xD;&#xA; &#xD;&#xA;&#xD;&#xA;The linked RELNR_depot numbers must be also grouped per group of Active and Inactive on a BASE record. So in case of the above example policy 18413.006216 is marked as Active and Inactive. &#xD;&#xA;The next step is to see what are the linked RELNR_depot for the Active part and what are the RELNR_depot for the inactive part.&#xD;&#xA;Example&#xD;&#xA; &#xD;&#xA;&#xD;&#xA;Active Part&#xD;&#xA;= Use RELNR_depot of the record(s) with DEPOT_REL.TOESL_CAT_A_I in (A, AI) &#xD;&#xA;    (there can be more than 1 record)&#xD;&#xA;&#xD;&#xA;Inactive part&#xD;&#xA;=  Use RELNR_depot of the record(s) with DEPOT_REL.TOESL_CAT_A_I in (I, AI) &#xD;&#xA;    (there can be more than 1 record)   &#xD;&#xA;&#xD;&#xA;&#xD;&#xA;Now that the RELNR_depot numbers are known, on a policy level this can be calculated (be aware that there can be multiple RELNR_depot numbers): &#xD;&#xA;&#xD;&#xA;&lt;&lt;Met Bart bespreken: gaan wij eerst per depotnr de bedragen ophalen en daarna per polis bepalen welke depotnummers er voor active zijn en dan de berekening voor het aantal jaren berekenen en dan hetzelfde voor als er op de polis ook inactive toeslag is>> &#xD;&#xA;&#xD;&#xA;For each RELNR_depot found: &#xD;&#xA;        Find the most recent record in RELATIE where &#xD;&#xA;        RELATIE.WIJZ_VERWERK_DAT &lt; 1-1-YYYY from BASE.REFERENCE_DATE&#xD;&#xA;        Calculate TMP_SALDO = ΣRELATIE.VRG_J_SLD_VERSCHUL for all RELNR_depot per Active/Inactive category &#xD;&#xA;&#xD;&#xA;        Find all records in Lego Journal (Z90_O939) where&#xD;&#xA;        JOURNAL.RELNR = Depot. RELNR_depot and&#xD;&#xA;        JOURNAL.BOEKCD not in (651, 652, 653, 654, 655, 656, 657) and&#xD;&#xA;        JOURNAL. BOEKJR = YYYY from BASE.REFERENCE_DATE -/- 1 &#xD;&#xA;        Calculate TMP_BOEK: ΣJOURNAL.BOEK for all records found&#xD;&#xA; &#xD;&#xA;    &#xD;&#xA; If TMP_BOEK > 0&#xD;&#xA;       &#xD;&#xA;        Calculate TMP_YEAR = (TMP_SALDO / TMP_BOEK) rounded down&#xD;&#xA;   &#xD;&#xA;      Use TMP_YEAR to fill PX.maxDuurIndexatie&#xD;&#xA;&#xD;&#xA;Deze berekening is dus per Active en Inactive!!!!!!!!!!!!!!!"/>
  </endUserTexts>
  <parameter xsi:type="Param:DerivedParameter" name="IP_REFERENCE_DATE" mandatory="false" multipleSelections="false">
    <endUserTexts label="IP_REFERENCE_DATE"/>
    <inlineType name="CHAR" primitiveType="CHAR"/>
    <defaultValue xsi:nil="true"/>
    <derivationRule>
      <procedureName>NNROOT.Z00_Applications.LEGO.UPO_MPO.DATALOADS::pr_reference_date</procedureName>
      <scriptObject>#/1</scriptObject>
    </derivationRule>
  </parameter>
  <parameter xsi:type="Param:DerivedParameter" name="IP_RUN_ID" mandatory="false" multipleSelections="false">
    <endUserTexts label="IP_RUN_ID"/>
    <inlineType name="CHAR" primitiveType="CHAR"/>
    <defaultValue xsi:nil="true"/>
    <derivationRule inputEnabled="false">
      <procedureName>NNROOT.Z00_Applications.LEGO.UPO_MPO.DATALOADS::pr_run_id_upo_mpo_lego</procedureName>
      <scriptObject>#/2</scriptObject>
    </derivationRule>
  </parameter>
  <executionHints/>
  <viewNode xsi:type="View:Aggregation" name="BASE">
    <endUserTexts label=" "/>
    <element name="RELNR">
      <endUserTexts label="RELNR"/>
      <inlineType primitiveType="NVARCHAR" length="5" precision="5" scale="0"/>
    </element>
    <element name="VOLGNR">
      <endUserTexts label="VOLGNR"/>
      <inlineType primitiveType="NVARCHAR" length="7" precision="7" scale="0"/>
    </element>
    <element name="VZ_CD_PA">
      <endUserTexts label="VZ_CD_PA"/>
      <inlineType primitiveType="NVARCHAR" length="4" precision="4" scale="0"/>
    </element>
    <element name="GRPNR">
      <inlineType primitiveType="NVARCHAR" length="3" precision="3" scale="0"/>
    </element>
    <element name="ACT_FIN_TYPE">
      <inlineType primitiveType="NVARCHAR" length="3" precision="3" scale="0"/>
    </element>
    <element name="ACT_TOESL_CAT">
      <inlineType primitiveType="NVARCHAR" length="3" precision="3" scale="0"/>
    </element>
    <element name="INACT_FIN_TYPE">
      <inlineType primitiveType="NVARCHAR" length="3" precision="3" scale="0"/>
    </element>
    <element name="INACT_TOESL_CAT">
      <inlineType primitiveType="NVARCHAR" length="3" precision="3" scale="0"/>
    </element>
    <element name="RUN_ID">
      <inlineType primitiveType="NVARCHAR" length="20" precision="20" scale="0"/>
    </element>
    <element name="RUN_TYPE">
      <inlineType primitiveType="NVARCHAR" length="1" precision="1" scale="0"/>
    </element>
    <element name="REFERENCE_DATE">
      <inlineType primitiveType="NVARCHAR" length="8" precision="8" scale="0"/>
    </element>
    <element name="RUN_TIMESTAMP">
      <inlineType primitiveType="NVARCHAR" length="21" precision="21" scale="0"/>
    </element>
    <element name="RUN_DATE">
      <inlineType primitiveType="NVARCHAR" length="8" precision="8" scale="0"/>
    </element>
    <input>
      <entity>#/0/NNROOT.Z00_Applications.LEGO.UPO_MPO.OUT::CV_OUT_Z21_O951_BASE</entity>
      <mapping xsi:type="Type:ElementMapping" targetName="RELNR" sourceName="RELNR"/>
      <mapping xsi:type="Type:ElementMapping" targetName="VOLGNR" sourceName="VOLGNR"/>
      <mapping xsi:type="Type:ElementMapping" targetName="VZ_CD_PA" sourceName="VZ_CD_PA"/>
      <mapping xsi:type="Type:ElementMapping" targetName="GRPNR" sourceName="GRPNR"/>
      <mapping xsi:type="Type:ElementMapping" targetName="ACT_FIN_TYPE" sourceName="ACT_FIN_TYPE"/>
      <mapping xsi:type="Type:ElementMapping" targetName="ACT_TOESL_CAT" sourceName="ACT_TOESL_CAT"/>
      <mapping xsi:type="Type:ElementMapping" targetName="INACT_FIN_TYPE" sourceName="INACT_FIN_TYPE"/>
      <mapping xsi:type="Type:ElementMapping" targetName="INACT_TOESL_CAT" sourceName="INACT_TOESL_CAT"/>
      <mapping xsi:type="Type:ElementMapping" targetName="RUN_ID" sourceName="RUN_ID"/>
      <mapping xsi:type="Type:ElementMapping" targetName="RUN_TYPE" sourceName="RUN_TYPE"/>
      <mapping xsi:type="Type:ElementMapping" targetName="REFERENCE_DATE" sourceName="REFERENCE_DATE"/>
      <mapping xsi:type="Type:ElementMapping" targetName="RUN_TIMESTAMP" sourceName="RUN_TIMESTAMP"/>
      <mapping xsi:type="Type:ElementMapping" targetName="RUN_DATE" sourceName="RUN_DATE"/>
      <parameterMapping xsi:type="Param:ParameterMapping" parameterNameOtherView="IP_RUN_ID" parameterName="IP_RUN_ID"/>
      <layout xCoordinate="70" yCoordinate="30"/>
    </input>
    <layout xCoordinate="284" yCoordinate="1151" width="-1" height="-1" expanded="true"/>
  </viewNode>
  <viewNode xsi:type="View:Projection" name="DEPOT_AMOUNTS">
    <endUserTexts label=" "/>
    <element name="JAAR">
      <inlineType primitiveType="NVARCHAR" length="4" precision="4" scale="0"/>
    </element>
    <element name="RELNR_DEPOT">
      <inlineType primitiveType="NVARCHAR" length="5" precision="5" scale="0"/>
    </element>
    <element name="SALDO_VJ">
      <endUserTexts label="SALDO_VJ"/>
      <inlineType primitiveType="DECIMAL" length="17" precision="17" scale="2"/>
    </element>
    <element name="REFERENCE_DATE" aggregationBehavior="NONE">
      <inlineType name="NVARCHAR" primitiveType="NVARCHAR" length="8"/>
      <calculationDefinition language="COLUMN_ENGINE">
        <formula>'$$IP_REFERENCE_DATE$$'</formula>
      </calculationDefinition>
    </element>
    <element name="REF_YEAR_MIN_1" aggregationBehavior="NONE">
      <inlineType name="NVARCHAR" primitiveType="NVARCHAR" length="4"/>
      <calculationDefinition language="COLUMN_ENGINE">
        <formula>string(int(leftstr(&quot;REFERENCE_DATE&quot;,4))-1)</formula>
      </calculationDefinition>
    </element>
    <filterExpression language="COLUMN_ENGINE">
      <formula>&quot;JAAR&quot; = &quot;REF_YEAR_MIN_1&quot;</formula>
    </filterExpression>
    <input>
      <entity>#/0/NNROOT.Z00_Applications.LEGO.UPO_MPO.DECOUPLING::CV_DEC_Z21_O944_DEPOTFINANCIERING_AMOUNTS</entity>
      <mapping xsi:type="Type:ElementMapping" targetName="JAAR" sourceName="JAAR"/>
      <mapping xsi:type="Type:ElementMapping" targetName="RELNR_DEPOT" sourceName="RELNR_DEPOT"/>
      <mapping xsi:type="Type:ElementMapping" targetName="SALDO_VJ" sourceName="SALDO_VJ"/>
      <layout xCoordinate="70" yCoordinate="30"/>
    </input>
    <layout xCoordinate="162" yCoordinate="1325" width="196" height="64" expanded="true"/>
  </viewNode>
  <viewNode xsi:type="View:Projection" name="DEPOT_RELATIES">
    <endUserTexts label=" "/>
    <element name="RELNR">
      <endUserTexts label="RELNR"/>
      <inlineType primitiveType="NVARCHAR" length="5" precision="5" scale="0"/>
    </element>
    <element name="TOESL_CAT_A_I">
      <inlineType primitiveType="NVARCHAR" length="2" precision="2" scale="0"/>
    </element>
    <element name="RELNR_DEPOT">
      <inlineType primitiveType="NVARCHAR" length="5" precision="5" scale="0"/>
    </element>
    <element name="GRPNR">
      <inlineType primitiveType="NVARCHAR" length="3" precision="3" scale="0"/>
    </element>
    <element name="ZVALIDFRM">
      <inlineType primitiveType="NVARCHAR" length="14" precision="14" scale="0"/>
    </element>
    <element name="IND_DEPOT_GRPNR_FILLED" aggregationBehavior="NONE">
      <inlineType name="NVARCHAR" primitiveType="NVARCHAR" length="1"/>
      <calculationDefinition language="COLUMN_ENGINE">
        <formula>if(&quot;GRPNR&quot;= '', '', 'X')</formula>
      </calculationDefinition>
    </element>
    <input>
      <entity>#/0/NNROOT.Z00_Applications.LEGO.UPO_MPO.DECOUPLING::CV_DEC_Z21_O943_DEPOTFINANCIERING_RELATIES</entity>
      <mapping xsi:type="Type:ElementMapping" targetName="RELNR" sourceName="RELNR"/>
      <mapping xsi:type="Type:ElementMapping" targetName="TOESL_CAT_A_I" sourceName="TOESL_CAT_DLN"/>
      <mapping xsi:type="Type:ElementMapping" targetName="RELNR_DEPOT" sourceName="RELNR_DEPOT"/>
      <mapping xsi:type="Type:ElementMapping" targetName="GRPNR" sourceName="GRPNR"/>
      <mapping xsi:type="Type:ElementMapping" targetName="ZVALIDFRM" sourceName="ZVALIDFRM"/>
      <parameterMapping xsi:type="Param:ParameterMapping" parameterNameOtherView="IP_REFERENCE_DATE" parameterName="IP_REFERENCE_DATE"/>
      <layout xCoordinate="70" yCoordinate="30"/>
    </input>
    <layout xCoordinate="340" yCoordinate="1499" width="205" height="64" expanded="true"/>
  </viewNode>
  <viewNode xsi:type="View:Aggregation" name="Aggregation_1">
    <endUserTexts label=" "/>
    <element name="ZVALIDFRM" aggregationBehavior="MAX">
      <inlineType primitiveType="NVARCHAR" length="14" precision="14" scale="0"/>
    </element>
    <input>
      <viewNode xsi:type="View:Projection">#/0/DEPOT_RELATIES</viewNode>
      <mapping xsi:type="Type:ElementMapping" targetName="ZVALIDFRM" sourceName="ZVALIDFRM"/>
      <layout xCoordinate="70" yCoordinate="30"/>
    </input>
    <layout xCoordinate="285" yCoordinate="1421" width="-1" height="-1" expanded="true"/>
  </viewNode>
  <viewNode xsi:type="View:JoinNode" name="Join_2" joinOrder="OUTSIDE_IN">
    <endUserTexts label=" "/>
    <element name="TOESL_CAT_A_I">
      <inlineType primitiveType="NVARCHAR" length="2" precision="2" scale="0"/>
    </element>
    <element name="RELNR_DEPOT">
      <inlineType primitiveType="NVARCHAR" length="5" precision="5" scale="0"/>
    </element>
    <element name="RELNR">
      <endUserTexts label="RELNR"/>
      <inlineType primitiveType="NVARCHAR" length="5" precision="5" scale="0"/>
    </element>
    <element name="GRPNR">
      <inlineType primitiveType="NVARCHAR" length="3" precision="3" scale="0"/>
    </element>
    <element name="IND_DEPOT_GRPNR_FILLED">
      <inlineType name="NVARCHAR" primitiveType="NVARCHAR" length="1" precision="0" scale="0"/>
    </element>
    <input>
      <viewNode xsi:type="View:Projection">#/0/DEPOT_RELATIES</viewNode>
      <mapping xsi:type="Type:ElementMapping" targetName="TOESL_CAT_A_I" sourceName="TOESL_CAT_A_I"/>
      <mapping xsi:type="Type:ElementMapping" targetName="RELNR_DEPOT" sourceName="RELNR_DEPOT"/>
      <mapping xsi:type="Type:ElementMapping" targetName="RELNR" sourceName="RELNR"/>
      <mapping xsi:type="Type:ElementMapping" targetName="GRPNR" sourceName="GRPNR"/>
      <mapping xsi:type="Type:ElementMapping" targetName="IND_DEPOT_GRPNR_FILLED" sourceName="IND_DEPOT_GRPNR_FILLED"/>
      <layout xCoordinate="50" yCoordinate="30"/>
    </input>
    <input>
      <viewNode xsi:type="View:Aggregation">#/0/Aggregation_1</viewNode>
      <layout xCoordinate="382" yCoordinate="30"/>
    </input>
    <layout xCoordinate="372" yCoordinate="1325" width="0" height="0" expanded="true"/>
    <join leftInput="#/0/Join_2/DEPOT_RELATIES" rightInput="#/0/Join_2/Aggregation_1" joinType="inner">
      <leftElementName>ZVALIDFRM</leftElementName>
      <rightElementName>ZVALIDFRM</rightElementName>
    </join>
  </viewNode>
  <viewNode xsi:type="View:JoinNode" name="Join_1" joinOrder="OUTSIDE_IN">
    <endUserTexts label=" "/>
    <element name="SALDO_VJ">
      <endUserTexts label="SALDO_VJ"/>
      <inlineType primitiveType="DECIMAL" length="17" precision="17" scale="2"/>
    </element>
    <element name="TOESL_CAT_A_I">
      <inlineType primitiveType="NVARCHAR" length="2" precision="2" scale="0"/>
    </element>
    <element name="RELNR_DEPOT">
      <inlineType primitiveType="NVARCHAR" length="5" precision="5" scale="0"/>
    </element>
    <element name="RELNR">
      <endUserTexts label="RELNR"/>
      <inlineType primitiveType="NVARCHAR" length="5" precision="5" scale="0"/>
    </element>
    <element name="GRPNR">
      <inlineType primitiveType="NVARCHAR" length="3" precision="3" scale="0"/>
    </element>
    <element name="IND_DEPOT_GRPNR_FILLED">
      <inlineType name="NVARCHAR" primitiveType="NVARCHAR" length="1" precision="0" scale="0"/>
    </element>
    <input>
      <viewNode xsi:type="View:JoinNode">#/0/Join_2</viewNode>
      <mapping xsi:type="Type:ElementMapping" targetName="TOESL_CAT_A_I" sourceName="TOESL_CAT_A_I"/>
      <mapping xsi:type="Type:ElementMapping" targetName="RELNR_DEPOT" sourceName="RELNR_DEPOT"/>
      <mapping xsi:type="Type:ElementMapping" targetName="RELNR" sourceName="RELNR"/>
      <mapping xsi:type="Type:ElementMapping" targetName="GRPNR" sourceName="GRPNR"/>
      <mapping xsi:type="Type:ElementMapping" targetName="IND_DEPOT_GRPNR_FILLED" sourceName="IND_DEPOT_GRPNR_FILLED"/>
      <layout xCoordinate="50" yCoordinate="30"/>
    </input>
    <input>
      <viewNode xsi:type="View:Projection">#/0/DEPOT_AMOUNTS</viewNode>
      <mapping xsi:type="Type:ElementMapping" targetName="SALDO_VJ" sourceName="SALDO_VJ"/>
      <layout xCoordinate="382" yCoordinate="30"/>
    </input>
    <layout xCoordinate="281" yCoordinate="1229" width="0" height="0" expanded="true"/>
    <join leftInput="#/0/Join_1/Join_2" rightInput="#/0/Join_1/DEPOT_AMOUNTS" joinType="inner">
      <leftElementName>RELNR_DEPOT</leftElementName>
      <rightElementName>RELNR_DEPOT</rightElementName>
    </join>
  </viewNode>
  <viewNode xsi:type="View:Projection" name="IND_GRPNR_X">
    <endUserTexts label=" "/>
    <element name="TOESL_CAT_A_I">
      <inlineType primitiveType="NVARCHAR" length="2" precision="2" scale="0"/>
    </element>
    <element name="RELNR_DEPOT">
      <inlineType primitiveType="NVARCHAR" length="5" precision="5" scale="0"/>
    </element>
    <element name="RELNR">
      <endUserTexts label="RELNR"/>
      <inlineType primitiveType="NVARCHAR" length="5" precision="5" scale="0"/>
    </element>
    <element name="GRPNR">
      <inlineType primitiveType="NVARCHAR" length="3" precision="3" scale="0"/>
    </element>
    <element name="IND_DEPOT_GRPNR_FILLED">
      <inlineType name="NVARCHAR" primitiveType="NVARCHAR" length="1" precision="0" scale="0"/>
    </element>
    <element name="SALDO_VJ">
      <endUserTexts label="SALDO_VJ"/>
      <inlineType primitiveType="DECIMAL" length="17" precision="17" scale="2"/>
    </element>
    <elementFilter elementName="IND_DEPOT_GRPNR_FILLED">
      <valueFilter xsi:type="Column:SingleValueFilter" including="true" value="X"/>
    </elementFilter>
    <input>
      <viewNode xsi:type="View:JoinNode">#/0/Join_1</viewNode>
      <mapping xsi:type="Type:ElementMapping" targetName="TOESL_CAT_A_I" sourceName="TOESL_CAT_A_I"/>
      <mapping xsi:type="Type:ElementMapping" targetName="RELNR_DEPOT" sourceName="RELNR_DEPOT"/>
      <mapping xsi:type="Type:ElementMapping" targetName="RELNR" sourceName="RELNR"/>
      <mapping xsi:type="Type:ElementMapping" targetName="GRPNR" sourceName="GRPNR"/>
      <mapping xsi:type="Type:ElementMapping" targetName="IND_DEPOT_GRPNR_FILLED" sourceName="IND_DEPOT_GRPNR_FILLED"/>
      <mapping xsi:type="Type:ElementMapping" targetName="SALDO_VJ" sourceName="SALDO_VJ"/>
      <layout xCoordinate="70" yCoordinate="30"/>
    </input>
    <layout xCoordinate="438" yCoordinate="1151" width="230" height="64" expanded="true"/>
  </viewNode>
  <viewNode xsi:type="View:JoinNode" name="Join_Base_Depot_GRPNR" joinOrder="OUTSIDE_IN">
    <endUserTexts label=" "/>
    <element name="RELNR">
      <endUserTexts label="RELNR"/>
      <inlineType primitiveType="NVARCHAR" length="5" precision="5" scale="0"/>
    </element>
    <element name="VOLGNR">
      <endUserTexts label="VOLGNR"/>
      <inlineType primitiveType="NVARCHAR" length="7" precision="7" scale="0"/>
    </element>
    <element name="VZ_CD_PA">
      <endUserTexts label="VZ_CD_PA"/>
      <inlineType primitiveType="NVARCHAR" length="4" precision="4" scale="0"/>
    </element>
    <element name="GRPNR">
      <inlineType primitiveType="NVARCHAR" length="3" precision="3" scale="0"/>
    </element>
    <element name="ACT_FIN_TYPE">
      <inlineType primitiveType="NVARCHAR" length="3" precision="3" scale="0"/>
    </element>
    <element name="ACT_TOESL_CAT">
      <inlineType primitiveType="NVARCHAR" length="3" precision="3" scale="0"/>
    </element>
    <element name="INACT_FIN_TYPE">
      <inlineType primitiveType="NVARCHAR" length="3" precision="3" scale="0"/>
    </element>
    <element name="INACT_TOESL_CAT">
      <inlineType primitiveType="NVARCHAR" length="3" precision="3" scale="0"/>
    </element>
    <element name="RUN_ID">
      <inlineType primitiveType="NVARCHAR" length="20" precision="20" scale="0"/>
    </element>
    <element name="RUN_TYPE">
      <inlineType primitiveType="NVARCHAR" length="1" precision="1" scale="0"/>
    </element>
    <element name="REFERENCE_DATE">
      <inlineType primitiveType="NVARCHAR" length="8" precision="8" scale="0"/>
    </element>
    <element name="RUN_TIMESTAMP">
      <inlineType primitiveType="NVARCHAR" length="21" precision="21" scale="0"/>
    </element>
    <element name="RUN_DATE">
      <inlineType primitiveType="NVARCHAR" length="8" precision="8" scale="0"/>
    </element>
    <element name="RELNR_FILE">
      <inlineType primitiveType="NVARCHAR" length="5" precision="5" scale="0"/>
    </element>
    <element name="TOESL_CAT_A_I_FILE">
      <inlineType primitiveType="NVARCHAR" length="2" precision="2" scale="0"/>
    </element>
    <element name="RELNR_DEPOT_FILE">
      <inlineType primitiveType="NVARCHAR" length="5" precision="5" scale="0"/>
    </element>
    <element name="IND_DEPOT_GRPNR_FILLED">
      <inlineType name="NVARCHAR" primitiveType="NVARCHAR" length="1" precision="0" scale="0"/>
    </element>
    <element name="SALDO_VJ">
      <endUserTexts label="SALDO_VJ"/>
      <inlineType primitiveType="DECIMAL" length="17" precision="17" scale="2"/>
    </element>
    <input>
      <viewNode xsi:type="View:Aggregation">#/0/BASE</viewNode>
      <mapping xsi:type="Type:ElementMapping" targetName="RELNR" sourceName="RELNR"/>
      <mapping xsi:type="Type:ElementMapping" targetName="VOLGNR" sourceName="VOLGNR"/>
      <mapping xsi:type="Type:ElementMapping" targetName="VZ_CD_PA" sourceName="VZ_CD_PA"/>
      <mapping xsi:type="Type:ElementMapping" targetName="GRPNR" sourceName="GRPNR"/>
      <mapping xsi:type="Type:ElementMapping" targetName="ACT_FIN_TYPE" sourceName="ACT_FIN_TYPE"/>
      <mapping xsi:type="Type:ElementMapping" targetName="ACT_TOESL_CAT" sourceName="ACT_TOESL_CAT"/>
      <mapping xsi:type="Type:ElementMapping" targetName="INACT_FIN_TYPE" sourceName="INACT_FIN_TYPE"/>
      <mapping xsi:type="Type:ElementMapping" targetName="INACT_TOESL_CAT" sourceName="INACT_TOESL_CAT"/>
      <mapping xsi:type="Type:ElementMapping" targetName="RUN_ID" sourceName="RUN_ID"/>
      <mapping xsi:type="Type:ElementMapping" targetName="RUN_TYPE" sourceName="RUN_TYPE"/>
      <mapping xsi:type="Type:ElementMapping" targetName="REFERENCE_DATE" sourceName="REFERENCE_DATE"/>
      <mapping xsi:type="Type:ElementMapping" targetName="RUN_TIMESTAMP" sourceName="RUN_TIMESTAMP"/>
      <mapping xsi:type="Type:ElementMapping" targetName="RUN_DATE" sourceName="RUN_DATE"/>
    </input>
    <input>
      <viewNode xsi:type="View:Projection">#/0/IND_GRPNR_X</viewNode>
      <mapping xsi:type="Type:ElementMapping" targetName="RELNR_FILE" sourceName="RELNR"/>
      <mapping xsi:type="Type:ElementMapping" targetName="TOESL_CAT_A_I_FILE" sourceName="TOESL_CAT_A_I"/>
      <mapping xsi:type="Type:ElementMapping" targetName="RELNR_DEPOT_FILE" sourceName="RELNR_DEPOT"/>
      <mapping xsi:type="Type:ElementMapping" targetName="IND_DEPOT_GRPNR_FILLED" sourceName="IND_DEPOT_GRPNR_FILLED"/>
      <mapping xsi:type="Type:ElementMapping" targetName="SALDO_VJ" sourceName="SALDO_VJ"/>
    </input>
    <layout xCoordinate="350" yCoordinate="1055" width="271" height="82" expanded="true"/>
    <join leftInput="#/0/Join_Base_Depot_GRPNR/BASE" rightInput="#/0/Join_Base_Depot_GRPNR/IND_GRPNR_X" joinType="inner" cardinality="C1_1">
      <leftElementName>RELNR</leftElementName>
      <leftElementName>GRPNR</leftElementName>
      <rightElementName>RELNR</rightElementName>
      <rightElementName>GRPNR</rightElementName>
    </join>
  </viewNode>
  <viewNode xsi:type="View:Projection" name="IND_GRPNR_BLANK">
    <endUserTexts label=" "/>
    <element name="TOESL_CAT_A_I">
      <inlineType primitiveType="NVARCHAR" length="2" precision="2" scale="0"/>
    </element>
    <element name="RELNR_DEPOT">
      <inlineType primitiveType="NVARCHAR" length="5" precision="5" scale="0"/>
    </element>
    <element name="RELNR">
      <endUserTexts label="RELNR"/>
      <inlineType primitiveType="NVARCHAR" length="5" precision="5" scale="0"/>
    </element>
    <element name="GRPNR">
      <inlineType primitiveType="NVARCHAR" length="3" precision="3" scale="0"/>
    </element>
    <element name="IND_DEPOT_GRPNR_FILLED">
      <inlineType name="NVARCHAR" primitiveType="NVARCHAR" length="1" precision="0" scale="0"/>
    </element>
    <element name="SALDO_VJ">
      <endUserTexts label="SALDO_VJ"/>
      <inlineType primitiveType="DECIMAL" length="17" precision="17" scale="2"/>
    </element>
    <filterExpression language="COLUMN_ENGINE">
      <formula>(&quot;IND_DEPOT_GRPNR_FILLED&quot; = '')</formula>
    </filterExpression>
    <input>
      <viewNode xsi:type="View:JoinNode">#/0/Join_1</viewNode>
      <mapping xsi:type="Type:ElementMapping" targetName="TOESL_CAT_A_I" sourceName="TOESL_CAT_A_I"/>
      <mapping xsi:type="Type:ElementMapping" targetName="RELNR_DEPOT" sourceName="RELNR_DEPOT"/>
      <mapping xsi:type="Type:ElementMapping" targetName="RELNR" sourceName="RELNR"/>
      <mapping xsi:type="Type:ElementMapping" targetName="GRPNR" sourceName="GRPNR"/>
      <mapping xsi:type="Type:ElementMapping" targetName="IND_DEPOT_GRPNR_FILLED" sourceName="IND_DEPOT_GRPNR_FILLED"/>
      <mapping xsi:type="Type:ElementMapping" targetName="SALDO_VJ" sourceName="SALDO_VJ"/>
    </input>
    <layout xCoordinate="22" yCoordinate="1151" width="248" height="64" expanded="true"/>
  </viewNode>
  <viewNode xsi:type="View:JoinNode" name="Join_Base_Depot_REL" joinOrder="OUTSIDE_IN">
    <endUserTexts label=" "/>
    <element name="RELNR">
      <endUserTexts label="RELNR"/>
      <inlineType primitiveType="NVARCHAR" length="5" precision="5" scale="0"/>
    </element>
    <element name="VOLGNR">
      <endUserTexts label="VOLGNR"/>
      <inlineType primitiveType="NVARCHAR" length="7" precision="7" scale="0"/>
    </element>
    <element name="VZ_CD_PA">
      <endUserTexts label="VZ_CD_PA"/>
      <inlineType primitiveType="NVARCHAR" length="4" precision="4" scale="0"/>
    </element>
    <element name="GRPNR">
      <inlineType primitiveType="NVARCHAR" length="3" precision="3" scale="0"/>
    </element>
    <element name="ACT_FIN_TYPE">
      <inlineType primitiveType="NVARCHAR" length="3" precision="3" scale="0"/>
    </element>
    <element name="ACT_TOESL_CAT">
      <inlineType primitiveType="NVARCHAR" length="3" precision="3" scale="0"/>
    </element>
    <element name="INACT_FIN_TYPE">
      <inlineType primitiveType="NVARCHAR" length="3" precision="3" scale="0"/>
    </element>
    <element name="INACT_TOESL_CAT">
      <inlineType primitiveType="NVARCHAR" length="3" precision="3" scale="0"/>
    </element>
    <element name="RUN_ID">
      <inlineType primitiveType="NVARCHAR" length="20" precision="20" scale="0"/>
    </element>
    <element name="RUN_TYPE">
      <inlineType primitiveType="NVARCHAR" length="1" precision="1" scale="0"/>
    </element>
    <element name="REFERENCE_DATE">
      <inlineType primitiveType="NVARCHAR" length="8" precision="8" scale="0"/>
    </element>
    <element name="RUN_TIMESTAMP">
      <inlineType primitiveType="NVARCHAR" length="21" precision="21" scale="0"/>
    </element>
    <element name="RUN_DATE">
      <inlineType primitiveType="NVARCHAR" length="8" precision="8" scale="0"/>
    </element>
    <element name="RELNR_FILE">
      <inlineType primitiveType="NVARCHAR" length="5" precision="5" scale="0"/>
    </element>
    <element name="TOESL_CAT_A_I_FILE">
      <inlineType primitiveType="NVARCHAR" length="2" precision="2" scale="0"/>
    </element>
    <element name="RELNR_DEPOT_FILE">
      <inlineType primitiveType="NVARCHAR" length="5" precision="5" scale="0"/>
    </element>
    <element name="IND_DEPOT_GRPNR_FILLED">
      <inlineType name="NVARCHAR" primitiveType="NVARCHAR" length="1" precision="0" scale="0"/>
    </element>
    <element name="SALDO_VJ">
      <endUserTexts label="SALDO_VJ"/>
      <inlineType primitiveType="DECIMAL" length="17" precision="17" scale="2"/>
    </element>
    <input>
      <viewNode xsi:type="View:Aggregation">#/0/BASE</viewNode>
      <mapping xsi:type="Type:ElementMapping" targetName="RELNR" sourceName="RELNR"/>
      <mapping xsi:type="Type:ElementMapping" targetName="VOLGNR" sourceName="VOLGNR"/>
      <mapping xsi:type="Type:ElementMapping" targetName="VZ_CD_PA" sourceName="VZ_CD_PA"/>
      <mapping xsi:type="Type:ElementMapping" targetName="GRPNR" sourceName="GRPNR"/>
      <mapping xsi:type="Type:ElementMapping" targetName="ACT_FIN_TYPE" sourceName="ACT_FIN_TYPE"/>
      <mapping xsi:type="Type:ElementMapping" targetName="ACT_TOESL_CAT" sourceName="ACT_TOESL_CAT"/>
      <mapping xsi:type="Type:ElementMapping" targetName="INACT_FIN_TYPE" sourceName="INACT_FIN_TYPE"/>
      <mapping xsi:type="Type:ElementMapping" targetName="INACT_TOESL_CAT" sourceName="INACT_TOESL_CAT"/>
      <mapping xsi:type="Type:ElementMapping" targetName="RUN_ID" sourceName="RUN_ID"/>
      <mapping xsi:type="Type:ElementMapping" targetName="RUN_TYPE" sourceName="RUN_TYPE"/>
      <mapping xsi:type="Type:ElementMapping" targetName="REFERENCE_DATE" sourceName="REFERENCE_DATE"/>
      <mapping xsi:type="Type:ElementMapping" targetName="RUN_TIMESTAMP" sourceName="RUN_TIMESTAMP"/>
      <mapping xsi:type="Type:ElementMapping" targetName="RUN_DATE" sourceName="RUN_DATE"/>
    </input>
    <input>
      <viewNode xsi:type="View:Projection">#/0/IND_GRPNR_BLANK</viewNode>
      <mapping xsi:type="Type:ElementMapping" targetName="RELNR_FILE" sourceName="RELNR"/>
      <mapping xsi:type="Type:ElementMapping" targetName="TOESL_CAT_A_I_FILE" sourceName="TOESL_CAT_A_I"/>
      <mapping xsi:type="Type:ElementMapping" targetName="RELNR_DEPOT_FILE" sourceName="RELNR_DEPOT"/>
      <mapping xsi:type="Type:ElementMapping" targetName="IND_DEPOT_GRPNR_FILLED" sourceName="IND_DEPOT_GRPNR_FILLED"/>
      <mapping xsi:type="Type:ElementMapping" targetName="SALDO_VJ" sourceName="SALDO_VJ"/>
    </input>
    <layout xCoordinate="94" yCoordinate="1055" width="242" height="82" expanded="true"/>
    <join leftInput="#/0/Join_Base_Depot_REL/BASE" rightInput="#/0/Join_Base_Depot_REL/IND_GRPNR_BLANK" joinType="inner" cardinality="C1_1">
      <leftElementName>RELNR</leftElementName>
      <rightElementName>RELNR</rightElementName>
    </join>
  </viewNode>
  <viewNode xsi:type="View:Union" name="Union_1">
    <endUserTexts label=" "/>
    <element name="RELNR">
      <endUserTexts label="RELNR"/>
      <inlineType primitiveType="NVARCHAR" length="5" precision="5" scale="0"/>
    </element>
    <element name="VOLGNR">
      <endUserTexts label="VOLGNR"/>
      <inlineType primitiveType="NVARCHAR" length="7" precision="7" scale="0"/>
    </element>
    <element name="VZ_CD_PA">
      <endUserTexts label="VZ_CD_PA"/>
      <inlineType primitiveType="NVARCHAR" length="4" precision="4" scale="0"/>
    </element>
    <element name="GRPNR">
      <inlineType primitiveType="NVARCHAR" length="3" precision="3" scale="0"/>
    </element>
    <element name="ACT_FIN_TYPE">
      <inlineType primitiveType="NVARCHAR" length="3" precision="3" scale="0"/>
    </element>
    <element name="ACT_TOESL_CAT">
      <inlineType primitiveType="NVARCHAR" length="3" precision="3" scale="0"/>
    </element>
    <element name="INACT_FIN_TYPE">
      <inlineType primitiveType="NVARCHAR" length="3" precision="3" scale="0"/>
    </element>
    <element name="INACT_TOESL_CAT">
      <inlineType primitiveType="NVARCHAR" length="3" precision="3" scale="0"/>
    </element>
    <element name="RUN_ID">
      <inlineType primitiveType="NVARCHAR" length="20" precision="20" scale="0"/>
    </element>
    <element name="RUN_TYPE">
      <inlineType primitiveType="NVARCHAR" length="1" precision="1" scale="0"/>
    </element>
    <element name="REFERENCE_DATE">
      <inlineType primitiveType="NVARCHAR" length="8" precision="8" scale="0"/>
    </element>
    <element name="RUN_TIMESTAMP">
      <inlineType primitiveType="NVARCHAR" length="21" precision="21" scale="0"/>
    </element>
    <element name="RUN_DATE">
      <inlineType primitiveType="NVARCHAR" length="8" precision="8" scale="0"/>
    </element>
    <element name="RELNR_FILE">
      <inlineType primitiveType="NVARCHAR" length="5" precision="5" scale="0"/>
    </element>
    <element name="TOESL_CAT_A_I_FILE">
      <inlineType primitiveType="NVARCHAR" length="2" precision="2" scale="0"/>
    </element>
    <element name="RELNR_DEPOT_FILE">
      <inlineType primitiveType="NVARCHAR" length="5" precision="5" scale="0"/>
    </element>
    <element name="IND_DEPOT_GRPNR_FILLED" transparentFilter="false">
      <inlineType name="NVARCHAR" primitiveType="NVARCHAR" length="1" precision="0" scale="0"/>
    </element>
    <element name="SALDO_VJ" transparentFilter="false">
      <endUserTexts label="SALDO_VJ"/>
      <inlineType primitiveType="DECIMAL" length="17" precision="17" scale="2"/>
    </element>
    <input emptyUnionBehavior="NO_ROW">
      <viewNode xsi:type="View:JoinNode">#/0/Join_Base_Depot_REL</viewNode>
      <mapping xsi:type="Type:ElementMapping" targetName="RELNR" sourceName="RELNR"/>
      <mapping xsi:type="Type:ElementMapping" targetName="VOLGNR" sourceName="VOLGNR"/>
      <mapping xsi:type="Type:ElementMapping" targetName="VZ_CD_PA" sourceName="VZ_CD_PA"/>
      <mapping xsi:type="Type:ElementMapping" targetName="GRPNR" sourceName="GRPNR"/>
      <mapping xsi:type="Type:ElementMapping" targetName="ACT_FIN_TYPE" sourceName="ACT_FIN_TYPE"/>
      <mapping xsi:type="Type:ElementMapping" targetName="ACT_TOESL_CAT" sourceName="ACT_TOESL_CAT"/>
      <mapping xsi:type="Type:ElementMapping" targetName="INACT_FIN_TYPE" sourceName="INACT_FIN_TYPE"/>
      <mapping xsi:type="Type:ElementMapping" targetName="INACT_TOESL_CAT" sourceName="INACT_TOESL_CAT"/>
      <mapping xsi:type="Type:ElementMapping" targetName="RUN_ID" sourceName="RUN_ID"/>
      <mapping xsi:type="Type:ElementMapping" targetName="RUN_TYPE" sourceName="RUN_TYPE"/>
      <mapping xsi:type="Type:ElementMapping" targetName="REFERENCE_DATE" sourceName="REFERENCE_DATE"/>
      <mapping xsi:type="Type:ElementMapping" targetName="RUN_TIMESTAMP" sourceName="RUN_TIMESTAMP"/>
      <mapping xsi:type="Type:ElementMapping" targetName="RUN_DATE" sourceName="RUN_DATE"/>
      <mapping xsi:type="Type:ElementMapping" targetName="RELNR_FILE" sourceName="RELNR_FILE"/>
      <mapping xsi:type="Type:ElementMapping" targetName="TOESL_CAT_A_I_FILE" sourceName="TOESL_CAT_A_I_FILE"/>
      <mapping xsi:type="Type:ElementMapping" targetName="RELNR_DEPOT_FILE" sourceName="RELNR_DEPOT_FILE"/>
      <mapping xsi:type="Type:ElementMapping" targetName="IND_DEPOT_GRPNR_FILLED" sourceName="IND_DEPOT_GRPNR_FILLED"/>
      <mapping xsi:type="Type:ElementMapping" targetName="SALDO_VJ" sourceName="SALDO_VJ"/>
    </input>
    <input emptyUnionBehavior="NO_ROW">
      <viewNode xsi:type="View:JoinNode">#/0/Join_Base_Depot_GRPNR</viewNode>
      <mapping xsi:type="Type:ElementMapping" targetName="RELNR" sourceName="RELNR"/>
      <mapping xsi:type="Type:ElementMapping" targetName="VOLGNR" sourceName="VOLGNR"/>
      <mapping xsi:type="Type:ElementMapping" targetName="VZ_CD_PA" sourceName="VZ_CD_PA"/>
      <mapping xsi:type="Type:ElementMapping" targetName="GRPNR" sourceName="GRPNR"/>
      <mapping xsi:type="Type:ElementMapping" targetName="ACT_FIN_TYPE" sourceName="ACT_FIN_TYPE"/>
      <mapping xsi:type="Type:ElementMapping" targetName="ACT_TOESL_CAT" sourceName="ACT_TOESL_CAT"/>
      <mapping xsi:type="Type:ElementMapping" targetName="INACT_FIN_TYPE" sourceName="INACT_FIN_TYPE"/>
      <mapping xsi:type="Type:ElementMapping" targetName="INACT_TOESL_CAT" sourceName="INACT_TOESL_CAT"/>
      <mapping xsi:type="Type:ElementMapping" targetName="RUN_ID" sourceName="RUN_ID"/>
      <mapping xsi:type="Type:ElementMapping" targetName="RUN_TYPE" sourceName="RUN_TYPE"/>
      <mapping xsi:type="Type:ElementMapping" targetName="REFERENCE_DATE" sourceName="REFERENCE_DATE"/>
      <mapping xsi:type="Type:ElementMapping" targetName="RUN_TIMESTAMP" sourceName="RUN_TIMESTAMP"/>
      <mapping xsi:type="Type:ElementMapping" targetName="RUN_DATE" sourceName="RUN_DATE"/>
      <mapping xsi:type="Type:ElementMapping" targetName="RELNR_FILE" sourceName="RELNR_FILE"/>
      <mapping xsi:type="Type:ElementMapping" targetName="TOESL_CAT_A_I_FILE" sourceName="TOESL_CAT_A_I_FILE"/>
      <mapping xsi:type="Type:ElementMapping" targetName="RELNR_DEPOT_FILE" sourceName="RELNR_DEPOT_FILE"/>
      <mapping xsi:type="Type:ElementMapping" targetName="IND_DEPOT_GRPNR_FILLED" sourceName="IND_DEPOT_GRPNR_FILLED"/>
      <mapping xsi:type="Type:ElementMapping" targetName="SALDO_VJ" sourceName="SALDO_VJ"/>
    </input>
    <layout xCoordinate="274" yCoordinate="939" width="0" height="0" expanded="true"/>
  </viewNode>
  <viewNode xsi:type="View:Projection" name="IND_DEPOT_REL">
    <endUserTexts label=" "/>
    <element name="RELNR">
      <endUserTexts label="RELNR"/>
      <inlineType primitiveType="NVARCHAR" length="5" precision="5" scale="0"/>
    </element>
    <element name="VOLGNR">
      <endUserTexts label="VOLGNR"/>
      <inlineType primitiveType="NVARCHAR" length="7" precision="7" scale="0"/>
    </element>
    <element name="VZ_CD_PA">
      <endUserTexts label="VZ_CD_PA"/>
      <inlineType primitiveType="NVARCHAR" length="4" precision="4" scale="0"/>
    </element>
    <element name="GRPNR">
      <inlineType primitiveType="NVARCHAR" length="3" precision="3" scale="0"/>
    </element>
    <element name="ACT_FIN_TYPE">
      <inlineType primitiveType="NVARCHAR" length="3" precision="3" scale="0"/>
    </element>
    <element name="ACT_TOESL_CAT">
      <inlineType primitiveType="NVARCHAR" length="3" precision="3" scale="0"/>
    </element>
    <element name="INACT_FIN_TYPE">
      <inlineType primitiveType="NVARCHAR" length="3" precision="3" scale="0"/>
    </element>
    <element name="INACT_TOESL_CAT">
      <inlineType primitiveType="NVARCHAR" length="3" precision="3" scale="0"/>
    </element>
    <element name="RUN_ID">
      <inlineType primitiveType="NVARCHAR" length="20" precision="20" scale="0"/>
    </element>
    <element name="RUN_TYPE">
      <inlineType primitiveType="NVARCHAR" length="1" precision="1" scale="0"/>
    </element>
    <element name="REFERENCE_DATE">
      <inlineType primitiveType="NVARCHAR" length="8" precision="8" scale="0"/>
    </element>
    <element name="RUN_TIMESTAMP">
      <inlineType primitiveType="NVARCHAR" length="21" precision="21" scale="0"/>
    </element>
    <element name="RUN_DATE">
      <inlineType primitiveType="NVARCHAR" length="8" precision="8" scale="0"/>
    </element>
    <element name="RELNR_FILE">
      <inlineType primitiveType="NVARCHAR" length="5" precision="5" scale="0"/>
    </element>
    <element name="TOESL_CAT_A_I_FILE">
      <inlineType primitiveType="NVARCHAR" length="2" precision="2" scale="0"/>
    </element>
    <element name="RELNR_DEPOT_FILE">
      <inlineType primitiveType="NVARCHAR" length="5" precision="5" scale="0"/>
    </element>
    <element name="IND_DEPOT_GRPNR_FILLED" transparentFilter="false">
      <inlineType name="NVARCHAR" primitiveType="NVARCHAR" length="1" precision="0" scale="0"/>
    </element>
    <element name="SALDO_VJ" transparentFilter="false">
      <endUserTexts label="SALDO_VJ"/>
      <inlineType primitiveType="DECIMAL" length="17" precision="17" scale="2"/>
    </element>
    <element name="ACT_IND_BASE" aggregationBehavior="NONE">
      <inlineType name="NVARCHAR" primitiveType="NVARCHAR" length="1"/>
      <calculationDefinition language="SQL">
        <formula>CASE WHEN &quot;ACT_FIN_TYPE&quot; IN ('3','7', '8') AND &quot;ACT_TOESL_CAT&quot; IN ('3', '4')&#xD;
THEN 'X'&#xD;
ELSE ''&#xD;
END</formula>
      </calculationDefinition>
    </element>
    <element name="INACT_IND_BASE" aggregationBehavior="NONE">
      <inlineType name="NVARCHAR" primitiveType="NVARCHAR" length="1"/>
      <calculationDefinition language="SQL">
        <formula>CASE WHEN &quot;INACT_FIN_TYPE&quot; IN ('3','7', '8') AND &quot;INACT_TOESL_CAT&quot; IN ('3', '4')&#xD;
THEN 'X'&#xD;
ELSE ''&#xD;
END</formula>
      </calculationDefinition>
    </element>
    <element name="REF_YEAR_0101" aggregationBehavior="NONE">
      <inlineType name="NVARCHAR" primitiveType="NVARCHAR" length="8"/>
      <calculationDefinition language="SQL">
        <formula>left(&quot;REFERENCE_DATE&quot;,4) || '0101'</formula>
      </calculationDefinition>
    </element>
    <element name="ACT_INACT_IND_RELNR_DEPOT" aggregationBehavior="NONE">
      <inlineType name="NVARCHAR" primitiveType="NVARCHAR" length="2"/>
      <calculationDefinition language="SQL">
        <formula>CASE WHEN ((&quot;ACT_FIN_TYPE&quot; IN ('3','7', '8') AND &quot;ACT_TOESL_CAT&quot; IN ('3', '4') AND &quot;TOESL_CAT_A_I_FILE&quot; IN ('AI')) AND (&quot;INACT_FIN_TYPE&quot; IN ('3','7', '8') AND &quot;INACT_TOESL_CAT&quot; IN ('3', '4') AND &quot;TOESL_CAT_A_I_FILE&quot; IN ('AI')))&#xD;
          THEN 'AI'            &#xD;
          WHEN &quot;ACT_FIN_TYPE&quot; IN ('3','7', '8') AND &quot;ACT_TOESL_CAT&quot; IN ('3', '4') AND &quot;TOESL_CAT_A_I_FILE&quot; IN ('A', 'AI')&#xD;
          THEN 'A'          &#xD;
          WHEN &quot;INACT_FIN_TYPE&quot; IN ('3','7', '8') AND &quot;INACT_TOESL_CAT&quot; IN ('3', '4') AND &quot;TOESL_CAT_A_I_FILE&quot; IN ('I', 'AI')&#xD;
          THEN 'I'  &#xD;
ELSE ''&#xD;
END</formula>
      </calculationDefinition>
    </element>
    <input>
      <viewNode xsi:type="View:Union">#/0/Union_1</viewNode>
      <mapping xsi:type="Type:ElementMapping" targetName="RELNR" sourceName="RELNR"/>
      <mapping xsi:type="Type:ElementMapping" targetName="VOLGNR" sourceName="VOLGNR"/>
      <mapping xsi:type="Type:ElementMapping" targetName="VZ_CD_PA" sourceName="VZ_CD_PA"/>
      <mapping xsi:type="Type:ElementMapping" targetName="GRPNR" sourceName="GRPNR"/>
      <mapping xsi:type="Type:ElementMapping" targetName="ACT_FIN_TYPE" sourceName="ACT_FIN_TYPE"/>
      <mapping xsi:type="Type:ElementMapping" targetName="ACT_TOESL_CAT" sourceName="ACT_TOESL_CAT"/>
      <mapping xsi:type="Type:ElementMapping" targetName="INACT_FIN_TYPE" sourceName="INACT_FIN_TYPE"/>
      <mapping xsi:type="Type:ElementMapping" targetName="INACT_TOESL_CAT" sourceName="INACT_TOESL_CAT"/>
      <mapping xsi:type="Type:ElementMapping" targetName="RUN_ID" sourceName="RUN_ID"/>
      <mapping xsi:type="Type:ElementMapping" targetName="RUN_TYPE" sourceName="RUN_TYPE"/>
      <mapping xsi:type="Type:ElementMapping" targetName="REFERENCE_DATE" sourceName="REFERENCE_DATE"/>
      <mapping xsi:type="Type:ElementMapping" targetName="RUN_TIMESTAMP" sourceName="RUN_TIMESTAMP"/>
      <mapping xsi:type="Type:ElementMapping" targetName="RUN_DATE" sourceName="RUN_DATE"/>
      <mapping xsi:type="Type:ElementMapping" targetName="RELNR_FILE" sourceName="RELNR_FILE"/>
      <mapping xsi:type="Type:ElementMapping" targetName="TOESL_CAT_A_I_FILE" sourceName="TOESL_CAT_A_I_FILE"/>
      <mapping xsi:type="Type:ElementMapping" targetName="RELNR_DEPOT_FILE" sourceName="RELNR_DEPOT_FILE"/>
      <mapping xsi:type="Type:ElementMapping" targetName="IND_DEPOT_GRPNR_FILLED" sourceName="IND_DEPOT_GRPNR_FILLED"/>
      <mapping xsi:type="Type:ElementMapping" targetName="SALDO_VJ" sourceName="SALDO_VJ"/>
    </input>
    <layout xCoordinate="243" yCoordinate="861" width="202" height="64" expanded="true"/>
  </viewNode>
  <viewNode xsi:type="View:Projection" name="RELATIE_MUT">
    <endUserTexts label=" "/>
    <element name="RELNR">
      <endUserTexts label="RELNR"/>
      <inlineType primitiveType="NVARCHAR" length="5" precision="5" scale="0"/>
    </element>
    <element name="WYZ_VERWERK_DAT">
      <inlineType primitiveType="NVARCHAR" length="8" precision="8" scale="0"/>
    </element>
    <element name="VRG_J_SLD_VERSCHUL">
      <inlineType primitiveType="DECIMAL" length="17" precision="17" scale="3"/>
    </element>
    <element name="DUMMY" aggregationBehavior="NONE">
      <inlineType name="INTEGER" primitiveType="INTEGER"/>
      <calculationDefinition language="COLUMN_ENGINE">
        <formula>1</formula>
      </calculationDefinition>
    </element>
    <input>
      <entity>#/0/NNROOT.Z00_Applications.LEGO.UPO_MPO.DECOUPLING::CV_DEC_Z90_O214_LEGO_RELATIE_MUT</entity>
      <mapping xsi:type="Type:ElementMapping" targetName="RELNR" sourceName="RELNR_PADDED"/>
      <mapping xsi:type="Type:ElementMapping" targetName="WYZ_VERWERK_DAT" sourceName="WYZ_VERWERK_DAT"/>
      <mapping xsi:type="Type:ElementMapping" targetName="VRG_J_SLD_VERSCHUL" sourceName="VRG_J_SLD_VERSCHUL"/>
    </input>
    <layout xCoordinate="483" yCoordinate="783" width="186" height="64" expanded="true"/>
  </viewNode>
  <viewNode xsi:type="View:Aggregation" name="ACT_INACT_IND_RELNR_DEPOT_NE_BLANK">
    <endUserTexts label=" "/>
    <element name="RELNR">
      <endUserTexts label="RELNR"/>
      <inlineType primitiveType="NVARCHAR" length="5" precision="5" scale="0"/>
    </element>
    <element name="VOLGNR">
      <endUserTexts label="VOLGNR"/>
      <inlineType primitiveType="NVARCHAR" length="7" precision="7" scale="0"/>
    </element>
    <element name="VZ_CD_PA">
      <endUserTexts label="VZ_CD_PA"/>
      <inlineType primitiveType="NVARCHAR" length="4" precision="4" scale="0"/>
    </element>
    <element name="GRPNR">
      <inlineType primitiveType="NVARCHAR" length="3" precision="3" scale="0"/>
    </element>
    <element name="ACT_FIN_TYPE">
      <inlineType primitiveType="NVARCHAR" length="3" precision="3" scale="0"/>
    </element>
    <element name="ACT_TOESL_CAT">
      <inlineType primitiveType="NVARCHAR" length="3" precision="3" scale="0"/>
    </element>
    <element name="INACT_FIN_TYPE">
      <inlineType primitiveType="NVARCHAR" length="3" precision="3" scale="0"/>
    </element>
    <element name="INACT_TOESL_CAT">
      <inlineType primitiveType="NVARCHAR" length="3" precision="3" scale="0"/>
    </element>
    <element name="ACT_IND_BASE">
      <inlineType name="NVARCHAR" primitiveType="NVARCHAR" length="1" precision="0" scale="0"/>
    </element>
    <element name="INACT_IND_BASE">
      <inlineType name="NVARCHAR" primitiveType="NVARCHAR" length="1" precision="0" scale="0"/>
    </element>
    <element name="RUN_ID">
      <inlineType primitiveType="NVARCHAR" length="20" precision="20" scale="0"/>
    </element>
    <element name="RUN_TYPE">
      <inlineType primitiveType="NVARCHAR" length="1" precision="1" scale="0"/>
    </element>
    <element name="RUN_TIMESTAMP">
      <inlineType primitiveType="NVARCHAR" length="21" precision="21" scale="0"/>
    </element>
    <element name="RUN_DATE">
      <inlineType primitiveType="NVARCHAR" length="8" precision="8" scale="0"/>
    </element>
    <element name="RELNR_FILE">
      <inlineType primitiveType="NVARCHAR" length="5" precision="5" scale="0"/>
    </element>
    <element name="TOESL_CAT_A_I_FILE">
      <inlineType primitiveType="NVARCHAR" length="2" precision="2" scale="0"/>
    </element>
    <element name="RELNR_DEPOT_FILE">
      <inlineType primitiveType="NVARCHAR" length="5" precision="5" scale="0"/>
    </element>
    <element name="IND_DEPOT_GRPNR_FILLED" transparentFilter="false">
      <inlineType name="NVARCHAR" primitiveType="NVARCHAR" length="1" precision="0" scale="0"/>
    </element>
    <element name="REF_YEAR_0101">
      <inlineType name="NVARCHAR" primitiveType="NVARCHAR" length="8" precision="0" scale="0"/>
    </element>
    <element name="REFERENCE_DATE">
      <inlineType primitiveType="NVARCHAR" length="8" precision="8" scale="0"/>
    </element>
    <element name="ACT_INACT_IND_RELNR_DEPOT">
      <inlineType name="NVARCHAR" primitiveType="NVARCHAR" length="2" precision="0" scale="0"/>
    </element>
    <element name="SALDO_VJ" transparentFilter="false">
      <endUserTexts label="SALDO_VJ"/>
      <inlineType primitiveType="DECIMAL" length="17" precision="17" scale="2"/>
    </element>
    <element name="REF_YEAR_MIN_1" aggregationBehavior="NONE">
      <inlineType name="NVARCHAR" primitiveType="NVARCHAR" length="4"/>
      <calculationDefinition language="SQL">
        <formula>to_int(left(&quot;REFERENCE_DATE&quot;,4)-1)</formula>
      </calculationDefinition>
    </element>
    <filterExpression language="COLUMN_ENGINE">
      <formula>&quot;ACT_INACT_IND_RELNR_DEPOT&quot; != ''</formula>
    </filterExpression>
    <input>
      <viewNode xsi:type="View:Projection">#/0/IND_DEPOT_REL</viewNode>
      <mapping xsi:type="Type:ElementMapping" targetName="RELNR" sourceName="RELNR"/>
      <mapping xsi:type="Type:ElementMapping" targetName="VOLGNR" sourceName="VOLGNR"/>
      <mapping xsi:type="Type:ElementMapping" targetName="VZ_CD_PA" sourceName="VZ_CD_PA"/>
      <mapping xsi:type="Type:ElementMapping" targetName="GRPNR" sourceName="GRPNR"/>
      <mapping xsi:type="Type:ElementMapping" targetName="ACT_FIN_TYPE" sourceName="ACT_FIN_TYPE"/>
      <mapping xsi:type="Type:ElementMapping" targetName="ACT_TOESL_CAT" sourceName="ACT_TOESL_CAT"/>
      <mapping xsi:type="Type:ElementMapping" targetName="INACT_FIN_TYPE" sourceName="INACT_FIN_TYPE"/>
      <mapping xsi:type="Type:ElementMapping" targetName="INACT_TOESL_CAT" sourceName="INACT_TOESL_CAT"/>
      <mapping xsi:type="Type:ElementMapping" targetName="ACT_IND_BASE" sourceName="ACT_IND_BASE"/>
      <mapping xsi:type="Type:ElementMapping" targetName="INACT_IND_BASE" sourceName="INACT_IND_BASE"/>
      <mapping xsi:type="Type:ElementMapping" targetName="RUN_ID" sourceName="RUN_ID"/>
      <mapping xsi:type="Type:ElementMapping" targetName="RUN_TYPE" sourceName="RUN_TYPE"/>
      <mapping xsi:type="Type:ElementMapping" targetName="RUN_TIMESTAMP" sourceName="RUN_TIMESTAMP"/>
      <mapping xsi:type="Type:ElementMapping" targetName="RUN_DATE" sourceName="RUN_DATE"/>
      <mapping xsi:type="Type:ElementMapping" targetName="RELNR_FILE" sourceName="RELNR_FILE"/>
      <mapping xsi:type="Type:ElementMapping" targetName="TOESL_CAT_A_I_FILE" sourceName="TOESL_CAT_A_I_FILE"/>
      <mapping xsi:type="Type:ElementMapping" targetName="RELNR_DEPOT_FILE" sourceName="RELNR_DEPOT_FILE"/>
      <mapping xsi:type="Type:ElementMapping" targetName="IND_DEPOT_GRPNR_FILLED" sourceName="IND_DEPOT_GRPNR_FILLED"/>
      <mapping xsi:type="Type:ElementMapping" targetName="REF_YEAR_0101" sourceName="REF_YEAR_0101"/>
      <mapping xsi:type="Type:ElementMapping" targetName="REFERENCE_DATE" sourceName="REFERENCE_DATE"/>
      <mapping xsi:type="Type:ElementMapping" targetName="ACT_INACT_IND_RELNR_DEPOT" sourceName="ACT_INACT_IND_RELNR_DEPOT"/>
      <mapping xsi:type="Type:ElementMapping" targetName="SALDO_VJ" sourceName="SALDO_VJ"/>
    </input>
    <layout xCoordinate="683" yCoordinate="783" width="413" height="64" expanded="true"/>
  </viewNode>
  <viewNode xsi:type="View:JoinNode" name="Join_DEPOT_MUT" joinOrder="OUTSIDE_IN">
    <endUserTexts label=" "/>
    <element name="RELNR">
      <endUserTexts label="RELNR"/>
      <inlineType primitiveType="NVARCHAR" length="5" precision="5" scale="0"/>
    </element>
    <element name="ACT_FIN_TYPE">
      <inlineType primitiveType="NVARCHAR" length="3" precision="3" scale="0"/>
    </element>
    <element name="ACT_TOESL_CAT">
      <inlineType primitiveType="NVARCHAR" length="3" precision="3" scale="0"/>
    </element>
    <element name="ACT_IND_BASE">
      <inlineType name="NVARCHAR" primitiveType="NVARCHAR" length="1" precision="0" scale="0"/>
    </element>
    <element name="INACT_FIN_TYPE">
      <inlineType primitiveType="NVARCHAR" length="3" precision="3" scale="0"/>
    </element>
    <element name="INACT_TOESL_CAT">
      <inlineType primitiveType="NVARCHAR" length="3" precision="3" scale="0"/>
    </element>
    <element name="INACT_IND_BASE">
      <inlineType name="NVARCHAR" primitiveType="NVARCHAR" length="1" precision="0" scale="0"/>
    </element>
    <element name="ACT_INACT_IND_RELNR_DEPOT">
      <inlineType name="NVARCHAR" primitiveType="NVARCHAR" length="2" precision="0" scale="0"/>
    </element>
    <element name="RELNR_FILE">
      <inlineType primitiveType="NVARCHAR" length="5" precision="5" scale="0"/>
    </element>
    <element name="TOESL_CAT_A_I_FILE">
      <inlineType primitiveType="NVARCHAR" length="2" precision="2" scale="0"/>
    </element>
    <element name="RELNR_DEPOT_FILE">
      <inlineType primitiveType="NVARCHAR" length="5" precision="5" scale="0"/>
    </element>
    <element name="REF_YEAR_0101">
      <inlineType name="NVARCHAR" primitiveType="NVARCHAR" length="8" precision="0" scale="0"/>
    </element>
    <element name="WYZ_VERWERK_DAT">
      <inlineType primitiveType="NVARCHAR" length="8" precision="8" scale="0"/>
    </element>
    <element name="VRG_J_SLD_VERSCHUL">
      <inlineType primitiveType="DECIMAL" length="17" precision="17" scale="3"/>
    </element>
    <element name="SALDO_VJ" transparentFilter="false">
      <endUserTexts label="SALDO_VJ"/>
      <inlineType primitiveType="DECIMAL" length="17" precision="17" scale="2"/>
    </element>
    <input>
      <viewNode xsi:type="View:Aggregation">#/0/ACT_INACT_IND_RELNR_DEPOT_NE_BLANK</viewNode>
      <mapping xsi:type="Type:ElementMapping" targetName="RELNR" sourceName="RELNR"/>
      <mapping xsi:type="Type:ElementMapping" targetName="ACT_FIN_TYPE" sourceName="ACT_FIN_TYPE"/>
      <mapping xsi:type="Type:ElementMapping" targetName="ACT_TOESL_CAT" sourceName="ACT_TOESL_CAT"/>
      <mapping xsi:type="Type:ElementMapping" targetName="ACT_IND_BASE" sourceName="ACT_IND_BASE"/>
      <mapping xsi:type="Type:ElementMapping" targetName="INACT_FIN_TYPE" sourceName="INACT_FIN_TYPE"/>
      <mapping xsi:type="Type:ElementMapping" targetName="INACT_TOESL_CAT" sourceName="INACT_TOESL_CAT"/>
      <mapping xsi:type="Type:ElementMapping" targetName="INACT_IND_BASE" sourceName="INACT_IND_BASE"/>
      <mapping xsi:type="Type:ElementMapping" targetName="ACT_INACT_IND_RELNR_DEPOT" sourceName="ACT_INACT_IND_RELNR_DEPOT"/>
      <mapping xsi:type="Type:ElementMapping" targetName="RELNR_FILE" sourceName="RELNR_FILE"/>
      <mapping xsi:type="Type:ElementMapping" targetName="TOESL_CAT_A_I_FILE" sourceName="TOESL_CAT_A_I_FILE"/>
      <mapping xsi:type="Type:ElementMapping" targetName="RELNR_DEPOT_FILE" sourceName="RELNR_DEPOT_FILE"/>
      <mapping xsi:type="Type:ElementMapping" targetName="REF_YEAR_0101" sourceName="REF_YEAR_0101"/>
      <mapping xsi:type="Type:ElementMapping" targetName="SALDO_VJ" sourceName="SALDO_VJ"/>
    </input>
    <input>
      <viewNode xsi:type="View:Projection">#/0/RELATIE_MUT</viewNode>
      <mapping xsi:type="Type:ElementMapping" targetName="WYZ_VERWERK_DAT" sourceName="WYZ_VERWERK_DAT"/>
      <mapping xsi:type="Type:ElementMapping" targetName="VRG_J_SLD_VERSCHUL" sourceName="VRG_J_SLD_VERSCHUL"/>
    </input>
    <layout xCoordinate="445" yCoordinate="687" width="262" height="82" expanded="true"/>
    <join leftInput="#/0/Join_DEPOT_MUT/ACT_INACT_IND_RELNR_DEPOT_NE_BLANK" rightInput="#/0/Join_DEPOT_MUT/RELATIE_MUT" joinType="inner">
      <leftElementName>RELNR_DEPOT_FILE</leftElementName>
      <rightElementName>RELNR</rightElementName>
    </join>
  </viewNode>
  <viewNode xsi:type="View:Aggregation" name="WIJZ_VERWERK_DAT_LT_REF_YEAR0101">
    <endUserTexts label=" "/>
    <element name="RELNR">
      <endUserTexts label="RELNR"/>
      <inlineType primitiveType="NVARCHAR" length="5" precision="5" scale="0"/>
    </element>
    <element name="RELNR_FILE">
      <inlineType primitiveType="NVARCHAR" length="5" precision="5" scale="0"/>
    </element>
    <element name="TOESL_CAT_A_I_FILE">
      <inlineType primitiveType="NVARCHAR" length="2" precision="2" scale="0"/>
    </element>
    <element name="RELNR_DEPOT_FILE">
      <inlineType primitiveType="NVARCHAR" length="5" precision="5" scale="0"/>
    </element>
    <element name="WYZ_VERWERK_DAT">
      <inlineType primitiveType="NVARCHAR" length="8" precision="8" scale="0"/>
    </element>
    <element name="VRG_J_SLD_VERSCHUL">
      <inlineType primitiveType="DECIMAL" length="17" precision="17" scale="3"/>
    </element>
    <element name="REF_YEAR_0101">
      <inlineType name="NVARCHAR" primitiveType="NVARCHAR" length="8" precision="0" scale="0"/>
    </element>
    <element name="ACT_INACT_IND_RELNR_DEPOT">
      <inlineType name="NVARCHAR" primitiveType="NVARCHAR" length="2" precision="0" scale="0"/>
    </element>
    <element name="SALDO_VJ" transparentFilter="false">
      <endUserTexts label="SALDO_VJ"/>
      <inlineType primitiveType="DECIMAL" length="17" precision="17" scale="2"/>
    </element>
    <filterExpression language="COLUMN_ENGINE">
      <formula>(&quot;WYZ_VERWERK_DAT&quot; &lt; &quot;REF_YEAR_0101&quot;)</formula>
    </filterExpression>
    <input>
      <viewNode xsi:type="View:JoinNode">#/0/Join_DEPOT_MUT</viewNode>
      <mapping xsi:type="Type:ElementMapping" targetName="RELNR" sourceName="RELNR"/>
      <mapping xsi:type="Type:ElementMapping" targetName="RELNR_FILE" sourceName="RELNR_FILE"/>
      <mapping xsi:type="Type:ElementMapping" targetName="TOESL_CAT_A_I_FILE" sourceName="TOESL_CAT_A_I_FILE"/>
      <mapping xsi:type="Type:ElementMapping" targetName="RELNR_DEPOT_FILE" sourceName="RELNR_DEPOT_FILE"/>
      <mapping xsi:type="Type:ElementMapping" targetName="WYZ_VERWERK_DAT" sourceName="WYZ_VERWERK_DAT"/>
      <mapping xsi:type="Type:ElementMapping" targetName="VRG_J_SLD_VERSCHUL" sourceName="VRG_J_SLD_VERSCHUL"/>
      <mapping xsi:type="Type:ElementMapping" targetName="REF_YEAR_0101" sourceName="REF_YEAR_0101"/>
      <mapping xsi:type="Type:ElementMapping" targetName="ACT_INACT_IND_RELNR_DEPOT" sourceName="ACT_INACT_IND_RELNR_DEPOT"/>
      <mapping xsi:type="Type:ElementMapping" targetName="SALDO_VJ" sourceName="SALDO_VJ"/>
    </input>
    <layout xCoordinate="362" yCoordinate="609" width="428" height="64" expanded="true"/>
  </viewNode>
  <viewNode xsi:type="View:Rank" name="Rank_most_recent_WYZ_VERWERK_DAT">
    <endUserTexts label=" "/>
    <element name="RELNR">
      <endUserTexts label="RELNR"/>
      <inlineType primitiveType="NVARCHAR" length="5" precision="5" scale="0"/>
    </element>
    <element name="RELNR_FILE">
      <inlineType primitiveType="NVARCHAR" length="5" precision="5" scale="0"/>
    </element>
    <element name="TOESL_CAT_A_I_FILE">
      <inlineType primitiveType="NVARCHAR" length="2" precision="2" scale="0"/>
    </element>
    <element name="RELNR_DEPOT_FILE">
      <inlineType primitiveType="NVARCHAR" length="5" precision="5" scale="0"/>
    </element>
    <element name="ACT_INACT_IND_RELNR_DEPOT">
      <inlineType name="NVARCHAR" primitiveType="NVARCHAR" length="2" precision="0" scale="0"/>
    </element>
    <element name="WYZ_VERWERK_DAT">
      <inlineType primitiveType="NVARCHAR" length="8" precision="8" scale="0"/>
    </element>
    <element name="VRG_J_SLD_VERSCHUL">
      <inlineType primitiveType="DECIMAL" length="17" precision="17" scale="3"/>
    </element>
    <element name="REF_YEAR_0101">
      <inlineType name="NVARCHAR" primitiveType="NVARCHAR" length="8" precision="0" scale="0"/>
    </element>
    <element name="SALDO_VJ" transparentFilter="false">
      <endUserTexts label="SALDO_VJ"/>
      <inlineType primitiveType="DECIMAL" length="17" precision="17" scale="2"/>
    </element>
    <input>
      <viewNode xsi:type="View:Aggregation">#/0/WIJZ_VERWERK_DAT_LT_REF_YEAR0101</viewNode>
      <mapping xsi:type="Type:ElementMapping" targetName="RELNR" sourceName="RELNR"/>
      <mapping xsi:type="Type:ElementMapping" targetName="RELNR_FILE" sourceName="RELNR_FILE"/>
      <mapping xsi:type="Type:ElementMapping" targetName="TOESL_CAT_A_I_FILE" sourceName="TOESL_CAT_A_I_FILE"/>
      <mapping xsi:type="Type:ElementMapping" targetName="RELNR_DEPOT_FILE" sourceName="RELNR_DEPOT_FILE"/>
      <mapping xsi:type="Type:ElementMapping" targetName="ACT_INACT_IND_RELNR_DEPOT" sourceName="ACT_INACT_IND_RELNR_DEPOT"/>
      <mapping xsi:type="Type:ElementMapping" targetName="WYZ_VERWERK_DAT" sourceName="WYZ_VERWERK_DAT"/>
      <mapping xsi:type="Type:ElementMapping" targetName="VRG_J_SLD_VERSCHUL" sourceName="VRG_J_SLD_VERSCHUL"/>
      <mapping xsi:type="Type:ElementMapping" targetName="REF_YEAR_0101" sourceName="REF_YEAR_0101"/>
      <mapping xsi:type="Type:ElementMapping" targetName="SALDO_VJ" sourceName="SALDO_VJ"/>
    </input>
    <layout xCoordinate="388" yCoordinate="513" width="406" height="64" expanded="true"/>
    <windowFunction dynamicPartitionElements="false">
      <partitionElement>#/0/Rank_most_recent_WYZ_VERWERK_DAT/RELNR</partitionElement>
      <partitionElement>#/0/Rank_most_recent_WYZ_VERWERK_DAT/RELNR_DEPOT_FILE</partitionElement>
      <partitionElement>#/0/Rank_most_recent_WYZ_VERWERK_DAT/ACT_INACT_IND_RELNR_DEPOT</partitionElement>
      <order byElement="#/0/Rank_most_recent_WYZ_VERWERK_DAT/WYZ_VERWERK_DAT" direction="DESC"/>
      <rankThreshold>
        <constantValue>1</constantValue>
      </rankThreshold>
    </windowFunction>
  </viewNode>
  <viewNode xsi:type="View:Aggregation" name="TMP_SALDO_SUM_VRG_J_SLD_VERSCHUL">
    <endUserTexts label=" "/>
    <element name="RELNR">
      <endUserTexts label="RELNR"/>
      <inlineType primitiveType="NVARCHAR" length="5" precision="5" scale="0"/>
    </element>
    <element name="ACT_INACT_IND_RELNR_DEPOT">
      <inlineType name="NVARCHAR" primitiveType="NVARCHAR" length="2" precision="0" scale="0"/>
    </element>
    <element name="VRG_J_SLD_VERSCHUL" aggregationBehavior="SUM">
      <inlineType primitiveType="DECIMAL" length="17" precision="17" scale="3"/>
    </element>
    <element name="SALDO_VJ" transparentFilter="false" aggregationBehavior="SUM">
      <endUserTexts label="SALDO_VJ"/>
      <inlineType primitiveType="DECIMAL" length="17" precision="17" scale="2"/>
    </element>
    <input>
      <viewNode xsi:type="View:Rank">#/0/Rank_most_recent_WYZ_VERWERK_DAT</viewNode>
      <mapping xsi:type="Type:ElementMapping" targetName="RELNR" sourceName="RELNR"/>
      <mapping xsi:type="Type:ElementMapping" targetName="ACT_INACT_IND_RELNR_DEPOT" sourceName="ACT_INACT_IND_RELNR_DEPOT"/>
      <mapping xsi:type="Type:ElementMapping" targetName="VRG_J_SLD_VERSCHUL" sourceName="VRG_J_SLD_VERSCHUL"/>
      <mapping xsi:type="Type:ElementMapping" targetName="SALDO_VJ" sourceName="SALDO_VJ"/>
    </input>
    <layout xCoordinate="428" yCoordinate="435" width="357" height="64" expanded="true"/>
  </viewNode>
  <viewNode xsi:type="View:Aggregation" name="Filter_Base">
    <endUserTexts label=" "/>
    <element name="RELNR">
      <endUserTexts label="RELNR"/>
      <inlineType primitiveType="NVARCHAR" length="5" precision="5" scale="0"/>
    </element>
    <element name="ACT_FIN_TYPE">
      <inlineType primitiveType="NVARCHAR" length="3" precision="3" scale="0"/>
    </element>
    <element name="ACT_TOESL_CAT">
      <inlineType primitiveType="NVARCHAR" length="3" precision="3" scale="0"/>
    </element>
    <element name="ACT_IND_BASE">
      <inlineType name="NVARCHAR" primitiveType="NVARCHAR" length="1" precision="0" scale="0"/>
    </element>
    <element name="INACT_FIN_TYPE">
      <inlineType primitiveType="NVARCHAR" length="3" precision="3" scale="0"/>
    </element>
    <element name="INACT_TOESL_CAT">
      <inlineType primitiveType="NVARCHAR" length="3" precision="3" scale="0"/>
    </element>
    <element name="INACT_IND_BASE">
      <inlineType name="NVARCHAR" primitiveType="NVARCHAR" length="1" precision="0" scale="0"/>
    </element>
    <element name="TOESL_CAT_A_I_FILE">
      <inlineType primitiveType="NVARCHAR" length="2" precision="2" scale="0"/>
    </element>
    <element name="RELNR_DEPOT_FILE">
      <inlineType primitiveType="NVARCHAR" length="5" precision="5" scale="0"/>
    </element>
    <element name="ACT_INACT_IND_RELNR_DEPOT">
      <inlineType name="NVARCHAR" primitiveType="NVARCHAR" length="2" precision="0" scale="0"/>
    </element>
    <element name="VOLGNR">
      <endUserTexts label="VOLGNR"/>
      <inlineType primitiveType="NVARCHAR" length="7" precision="7" scale="0"/>
    </element>
    <element name="VZ_CD_PA">
      <endUserTexts label="VZ_CD_PA"/>
      <inlineType primitiveType="NVARCHAR" length="4" precision="4" scale="0"/>
    </element>
    <element name="SALDO_VJ" transparentFilter="false">
      <endUserTexts label="SALDO_VJ"/>
      <inlineType primitiveType="DECIMAL" length="17" precision="17" scale="2"/>
    </element>
    <filterExpression language="COLUMN_ENGINE">
      <formula>&quot;ACT_INACT_IND_RELNR_DEPOT&quot; != ''</formula>
    </filterExpression>
    <input>
      <viewNode xsi:type="View:Projection">#/0/IND_DEPOT_REL</viewNode>
      <mapping xsi:type="Type:ElementMapping" targetName="RELNR" sourceName="RELNR"/>
      <mapping xsi:type="Type:ElementMapping" targetName="ACT_FIN_TYPE" sourceName="ACT_FIN_TYPE"/>
      <mapping xsi:type="Type:ElementMapping" targetName="ACT_TOESL_CAT" sourceName="ACT_TOESL_CAT"/>
      <mapping xsi:type="Type:ElementMapping" targetName="ACT_IND_BASE" sourceName="ACT_IND_BASE"/>
      <mapping xsi:type="Type:ElementMapping" targetName="INACT_FIN_TYPE" sourceName="INACT_FIN_TYPE"/>
      <mapping xsi:type="Type:ElementMapping" targetName="INACT_TOESL_CAT" sourceName="INACT_TOESL_CAT"/>
      <mapping xsi:type="Type:ElementMapping" targetName="INACT_IND_BASE" sourceName="INACT_IND_BASE"/>
      <mapping xsi:type="Type:ElementMapping" targetName="TOESL_CAT_A_I_FILE" sourceName="TOESL_CAT_A_I_FILE"/>
      <mapping xsi:type="Type:ElementMapping" targetName="RELNR_DEPOT_FILE" sourceName="RELNR_DEPOT_FILE"/>
      <mapping xsi:type="Type:ElementMapping" targetName="ACT_INACT_IND_RELNR_DEPOT" sourceName="ACT_INACT_IND_RELNR_DEPOT"/>
      <mapping xsi:type="Type:ElementMapping" targetName="VOLGNR" sourceName="VOLGNR"/>
      <mapping xsi:type="Type:ElementMapping" targetName="VZ_CD_PA" sourceName="VZ_CD_PA"/>
      <mapping xsi:type="Type:ElementMapping" targetName="SALDO_VJ" sourceName="SALDO_VJ"/>
    </input>
    <layout xCoordinate="274" yCoordinate="435" width="0" height="0" expanded="true"/>
  </viewNode>
  <viewNode xsi:type="View:JoinNode" name="Join_TMP_SALDO" joinOrder="OUTSIDE_IN">
    <endUserTexts label=" "/>
    <element name="TMP_SALDO">
      <inlineType primitiveType="DECIMAL" length="17" precision="17" scale="3"/>
    </element>
    <element name="SALDO_VJ" transparentFilter="false">
      <endUserTexts label="SALDO_VJ"/>
      <inlineType primitiveType="DECIMAL" length="17" precision="17" scale="2"/>
    </element>
    <element name="RELNR">
      <endUserTexts label="RELNR"/>
      <inlineType primitiveType="NVARCHAR" length="5" precision="5" scale="0"/>
    </element>
    <element name="ACT_FIN_TYPE">
      <inlineType primitiveType="NVARCHAR" length="3" precision="3" scale="0"/>
    </element>
    <element name="ACT_TOESL_CAT">
      <inlineType primitiveType="NVARCHAR" length="3" precision="3" scale="0"/>
    </element>
    <element name="ACT_IND_BASE">
      <inlineType name="NVARCHAR" primitiveType="NVARCHAR" length="1" precision="0" scale="0"/>
    </element>
    <element name="INACT_FIN_TYPE">
      <inlineType primitiveType="NVARCHAR" length="3" precision="3" scale="0"/>
    </element>
    <element name="INACT_TOESL_CAT">
      <inlineType primitiveType="NVARCHAR" length="3" precision="3" scale="0"/>
    </element>
    <element name="INACT_IND_BASE">
      <inlineType name="NVARCHAR" primitiveType="NVARCHAR" length="1" precision="0" scale="0"/>
    </element>
    <element name="TOESL_CAT_A_I_FILE">
      <inlineType primitiveType="NVARCHAR" length="2" precision="2" scale="0"/>
    </element>
    <element name="RELNR_DEPOT_FILE">
      <inlineType primitiveType="NVARCHAR" length="5" precision="5" scale="0"/>
    </element>
    <element name="ACT_INACT_IND_RELNR_DEPOT">
      <inlineType name="NVARCHAR" primitiveType="NVARCHAR" length="2" precision="0" scale="0"/>
    </element>
    <element name="VOLGNR">
      <endUserTexts label="VOLGNR"/>
      <inlineType primitiveType="NVARCHAR" length="7" precision="7" scale="0"/>
    </element>
    <element name="VZ_CD_PA">
      <endUserTexts label="VZ_CD_PA"/>
      <inlineType primitiveType="NVARCHAR" length="4" precision="4" scale="0"/>
    </element>
    <element name="SALDO_VJ_2" transparentFilter="false">
      <inlineType primitiveType="DECIMAL" length="17" precision="17" scale="2"/>
    </element>
    <input>
      <viewNode xsi:type="View:Aggregation">#/0/Filter_Base</viewNode>
      <mapping xsi:type="Type:ElementMapping" targetName="RELNR" sourceName="RELNR"/>
      <mapping xsi:type="Type:ElementMapping" targetName="ACT_FIN_TYPE" sourceName="ACT_FIN_TYPE"/>
      <mapping xsi:type="Type:ElementMapping" targetName="ACT_TOESL_CAT" sourceName="ACT_TOESL_CAT"/>
      <mapping xsi:type="Type:ElementMapping" targetName="ACT_IND_BASE" sourceName="ACT_IND_BASE"/>
      <mapping xsi:type="Type:ElementMapping" targetName="INACT_FIN_TYPE" sourceName="INACT_FIN_TYPE"/>
      <mapping xsi:type="Type:ElementMapping" targetName="INACT_TOESL_CAT" sourceName="INACT_TOESL_CAT"/>
      <mapping xsi:type="Type:ElementMapping" targetName="INACT_IND_BASE" sourceName="INACT_IND_BASE"/>
      <mapping xsi:type="Type:ElementMapping" targetName="TOESL_CAT_A_I_FILE" sourceName="TOESL_CAT_A_I_FILE"/>
      <mapping xsi:type="Type:ElementMapping" targetName="RELNR_DEPOT_FILE" sourceName="RELNR_DEPOT_FILE"/>
      <mapping xsi:type="Type:ElementMapping" targetName="ACT_INACT_IND_RELNR_DEPOT" sourceName="ACT_INACT_IND_RELNR_DEPOT"/>
      <mapping xsi:type="Type:ElementMapping" targetName="VOLGNR" sourceName="VOLGNR"/>
      <mapping xsi:type="Type:ElementMapping" targetName="VZ_CD_PA" sourceName="VZ_CD_PA"/>
      <mapping xsi:type="Type:ElementMapping" targetName="SALDO_VJ_2" sourceName="SALDO_VJ"/>
      <layout xCoordinate="50" yCoordinate="30"/>
    </input>
    <input>
      <viewNode xsi:type="View:Aggregation">#/0/TMP_SALDO_SUM_VRG_J_SLD_VERSCHUL</viewNode>
      <mapping xsi:type="Type:ElementMapping" targetName="TMP_SALDO" sourceName="VRG_J_SLD_VERSCHUL"/>
      <mapping xsi:type="Type:ElementMapping" targetName="SALDO_VJ" sourceName="SALDO_VJ"/>
      <layout xCoordinate="422" yCoordinate="30"/>
    </input>
    <layout xCoordinate="496" yCoordinate="339" width="220" height="82" expanded="true"/>
    <join leftInput="#/0/Join_TMP_SALDO/Filter_Base" rightInput="#/0/Join_TMP_SALDO/TMP_SALDO_SUM_VRG_J_SLD_VERSCHUL" joinType="inner" cardinality="C1_1" dynamic="false">
      <leftElementName>RELNR</leftElementName>
      <leftElementName>ACT_INACT_IND_RELNR_DEPOT</leftElementName>
      <rightElementName>RELNR</rightElementName>
      <rightElementName>ACT_INACT_IND_RELNR_DEPOT</rightElementName>
    </join>
  </viewNode>
  <viewNode xsi:type="View:Aggregation" name="JOURNAL_ACTUALS">
    <endUserTexts label=" "/>
    <element name="RELNR">
      <endUserTexts label="RELNR"/>
      <inlineType primitiveType="NVARCHAR" length="5" precision="5" scale="0"/>
    </element>
    <element name="BOEK_ORG">
      <inlineType primitiveType="NVARCHAR" length="13" precision="13" scale="0"/>
    </element>
    <element name="BOEKJR">
      <inlineType primitiveType="NVARCHAR" length="4" precision="4" scale="0"/>
    </element>
    <element name="BOEKCD">
      <inlineType primitiveType="NVARCHAR" length="3" precision="3" scale="0"/>
    </element>
    <element name="REF_YEAR">
      <inlineType primitiveType="NVARCHAR" length="4" precision="4" scale="0"/>
    </element>
    <element name="REF_YEAR_MINUS_1">
      <inlineType primitiveType="NVARCHAR" length="4" precision="4" scale="0"/>
    </element>
    <element name="BOEK_PRE" aggregationBehavior="NONE">
      <inlineType name="DECIMAL" primitiveType="DECIMAL" length="17" scale="2"/>
      <calculationDefinition language="SQL">
        <formula>to_decimal( replace(&quot;BOEK_ORG&quot;,',','.') )</formula>
      </calculationDefinition>
    </element>
    <filterExpression language="COLUMN_ENGINE">
      <formula>not in (&quot;BOEKCD&quot;,'651','652','653','654','655','656','657')</formula>
    </filterExpression>
    <input>
      <entity>#/0/NNROOT.Z00_Applications.LEGO.UPO_MPO.OUT::CV_OUT_Z21_O939_JOURNAL</entity>
      <mapping xsi:type="Type:ElementMapping" targetName="RELNR" sourceName="RELNR"/>
      <mapping xsi:type="Type:ElementMapping" targetName="BOEK_ORG" sourceName="BOEK"/>
      <mapping xsi:type="Type:ElementMapping" targetName="BOEKJR" sourceName="BOEKJR"/>
      <mapping xsi:type="Type:ElementMapping" targetName="BOEKCD" sourceName="BOEKCD"/>
      <mapping xsi:type="Type:ElementMapping" targetName="REF_YEAR" sourceName="REF_YEAR"/>
      <mapping xsi:type="Type:ElementMapping" targetName="REF_YEAR_MINUS_1" sourceName="REF_YEAR_MINUS_1"/>
      <parameterMapping xsi:type="Param:ParameterMapping" parameterNameOtherView="IP_REFERENCE_DATE" parameterName="IP_REFERENCE_DATE"/>
    </input>
    <layout xCoordinate="804" yCoordinate="609" width="231" height="64" expanded="true"/>
  </viewNode>
  <viewNode xsi:type="View:JoinNode" name="Join_DEPOT_JRN" joinOrder="OUTSIDE_IN">
    <endUserTexts label=" "/>
    <element name="RELNR">
      <endUserTexts label="RELNR"/>
      <inlineType primitiveType="NVARCHAR" length="5" precision="5" scale="0"/>
    </element>
    <element name="ACT_FIN_TYPE">
      <inlineType primitiveType="NVARCHAR" length="3" precision="3" scale="0"/>
    </element>
    <element name="ACT_TOESL_CAT">
      <inlineType primitiveType="NVARCHAR" length="3" precision="3" scale="0"/>
    </element>
    <element name="ACT_IND_BASE">
      <inlineType name="NVARCHAR" primitiveType="NVARCHAR" length="1" precision="0" scale="0"/>
    </element>
    <element name="INACT_FIN_TYPE">
      <inlineType primitiveType="NVARCHAR" length="3" precision="3" scale="0"/>
    </element>
    <element name="INACT_TOESL_CAT">
      <inlineType primitiveType="NVARCHAR" length="3" precision="3" scale="0"/>
    </element>
    <element name="INACT_IND_BASE">
      <inlineType name="NVARCHAR" primitiveType="NVARCHAR" length="1" precision="0" scale="0"/>
    </element>
    <element name="ACT_INACT_IND_RELNR_DEPOT">
      <inlineType name="NVARCHAR" primitiveType="NVARCHAR" length="2" precision="0" scale="0"/>
    </element>
    <element name="RELNR_FILE">
      <inlineType primitiveType="NVARCHAR" length="5" precision="5" scale="0"/>
    </element>
    <element name="TOESL_CAT_A_I_FILE">
      <inlineType primitiveType="NVARCHAR" length="2" precision="2" scale="0"/>
    </element>
    <element name="RELNR_DEPOT_FILE">
      <inlineType primitiveType="NVARCHAR" length="5" precision="5" scale="0"/>
    </element>
    <element name="BOEK_PRE">
      <inlineType name="DECIMAL" primitiveType="DECIMAL" length="17" precision="0" scale="2"/>
    </element>
    <input>
      <viewNode xsi:type="View:Aggregation">#/0/ACT_INACT_IND_RELNR_DEPOT_NE_BLANK</viewNode>
      <mapping xsi:type="Type:ElementMapping" targetName="RELNR" sourceName="RELNR"/>
      <mapping xsi:type="Type:ElementMapping" targetName="ACT_FIN_TYPE" sourceName="ACT_FIN_TYPE"/>
      <mapping xsi:type="Type:ElementMapping" targetName="ACT_TOESL_CAT" sourceName="ACT_TOESL_CAT"/>
      <mapping xsi:type="Type:ElementMapping" targetName="ACT_IND_BASE" sourceName="ACT_IND_BASE"/>
      <mapping xsi:type="Type:ElementMapping" targetName="INACT_FIN_TYPE" sourceName="INACT_FIN_TYPE"/>
      <mapping xsi:type="Type:ElementMapping" targetName="INACT_TOESL_CAT" sourceName="INACT_TOESL_CAT"/>
      <mapping xsi:type="Type:ElementMapping" targetName="INACT_IND_BASE" sourceName="INACT_IND_BASE"/>
      <mapping xsi:type="Type:ElementMapping" targetName="ACT_INACT_IND_RELNR_DEPOT" sourceName="ACT_INACT_IND_RELNR_DEPOT"/>
      <mapping xsi:type="Type:ElementMapping" targetName="RELNR_FILE" sourceName="RELNR_FILE"/>
      <mapping xsi:type="Type:ElementMapping" targetName="TOESL_CAT_A_I_FILE" sourceName="TOESL_CAT_A_I_FILE"/>
      <mapping xsi:type="Type:ElementMapping" targetName="RELNR_DEPOT_FILE" sourceName="RELNR_DEPOT_FILE"/>
    </input>
    <input>
      <viewNode xsi:type="View:Aggregation">#/0/JOURNAL_ACTUALS</viewNode>
      <mapping xsi:type="Type:ElementMapping" targetName="BOEK_PRE" sourceName="BOEK_PRE"/>
    </input>
    <layout xCoordinate="868" yCoordinate="513" width="235" height="82" expanded="true"/>
    <join leftInput="#/0/Join_DEPOT_JRN/ACT_INACT_IND_RELNR_DEPOT_NE_BLANK" rightInput="#/0/Join_DEPOT_JRN/JOURNAL_ACTUALS" joinType="inner" cardinality="C1_N">
      <leftElementName>RELNR_DEPOT_FILE</leftElementName>
      <rightElementName>RELNR</rightElementName>
    </join>
  </viewNode>
  <viewNode xsi:type="View:Aggregation" name="DEPOT_JOURNAL">
    <endUserTexts label=" "/>
    <element name="RELNR">
      <endUserTexts label="RELNR"/>
      <inlineType primitiveType="NVARCHAR" length="5" precision="5" scale="0"/>
    </element>
    <element name="RELNR_FILE">
      <inlineType primitiveType="NVARCHAR" length="5" precision="5" scale="0"/>
    </element>
    <element name="TOESL_CAT_A_I_FILE">
      <inlineType primitiveType="NVARCHAR" length="2" precision="2" scale="0"/>
    </element>
    <element name="RELNR_DEPOT_FILE">
      <inlineType primitiveType="NVARCHAR" length="5" precision="5" scale="0"/>
    </element>
    <element name="ACT_INACT_IND_RELNR_DEPOT">
      <inlineType name="NVARCHAR" primitiveType="NVARCHAR" length="2" precision="0" scale="0"/>
    </element>
    <element name="BOEK_PRE" aggregationBehavior="SUM">
      <inlineType name="DECIMAL" primitiveType="DECIMAL" length="17" precision="0" scale="2"/>
    </element>
    <input>
      <viewNode xsi:type="View:JoinNode">#/0/Join_DEPOT_JRN</viewNode>
      <mapping xsi:type="Type:ElementMapping" targetName="RELNR" sourceName="RELNR"/>
      <mapping xsi:type="Type:ElementMapping" targetName="RELNR_FILE" sourceName="RELNR_FILE"/>
      <mapping xsi:type="Type:ElementMapping" targetName="TOESL_CAT_A_I_FILE" sourceName="TOESL_CAT_A_I_FILE"/>
      <mapping xsi:type="Type:ElementMapping" targetName="RELNR_DEPOT_FILE" sourceName="RELNR_DEPOT_FILE"/>
      <mapping xsi:type="Type:ElementMapping" targetName="ACT_INACT_IND_RELNR_DEPOT" sourceName="ACT_INACT_IND_RELNR_DEPOT"/>
      <mapping xsi:type="Type:ElementMapping" targetName="BOEK_PRE" sourceName="BOEK_PRE"/>
    </input>
    <layout xCoordinate="816" yCoordinate="435" width="209" height="64" expanded="true"/>
  </viewNode>
  <viewNode xsi:type="View:Aggregation" name="TMP_BOEK_SUM_BOEK">
    <endUserTexts label=" "/>
    <element name="RELNR">
      <endUserTexts label="RELNR"/>
      <inlineType primitiveType="NVARCHAR" length="5" precision="5" scale="0"/>
    </element>
    <element name="ACT_INACT_IND_RELNR_DEPOT">
      <inlineType name="NVARCHAR" primitiveType="NVARCHAR" length="2" precision="0" scale="0"/>
    </element>
    <element name="BOEK_PRE" aggregationBehavior="SUM">
      <inlineType name="DECIMAL" primitiveType="DECIMAL" length="17" precision="0" scale="2"/>
    </element>
    <input>
      <viewNode xsi:type="View:Aggregation">#/0/DEPOT_JOURNAL</viewNode>
      <mapping xsi:type="Type:ElementMapping" targetName="RELNR" sourceName="RELNR"/>
      <mapping xsi:type="Type:ElementMapping" targetName="ACT_INACT_IND_RELNR_DEPOT" sourceName="ACT_INACT_IND_RELNR_DEPOT"/>
      <mapping xsi:type="Type:ElementMapping" targetName="BOEK_PRE" sourceName="BOEK_PRE"/>
    </input>
    <layout xCoordinate="730" yCoordinate="339" width="251" height="64" expanded="true"/>
  </viewNode>
  <viewNode xsi:type="View:JoinNode" name="Join_TMP_BOEK" joinOrder="OUTSIDE_IN">
    <endUserTexts label=" "/>
    <element name="RELNR">
      <endUserTexts label="RELNR"/>
      <inlineType primitiveType="NVARCHAR" length="5" precision="5" scale="0"/>
    </element>
    <element name="VOLGNR">
      <endUserTexts label="VOLGNR"/>
      <inlineType primitiveType="NVARCHAR" length="7" precision="7" scale="0"/>
    </element>
    <element name="VZ_CD_PA">
      <endUserTexts label="VZ_CD_PA"/>
      <inlineType primitiveType="NVARCHAR" length="4" precision="4" scale="0"/>
    </element>
    <element name="ACT_FIN_TYPE">
      <inlineType primitiveType="NVARCHAR" length="3" precision="3" scale="0"/>
    </element>
    <element name="ACT_TOESL_CAT">
      <inlineType primitiveType="NVARCHAR" length="3" precision="3" scale="0"/>
    </element>
    <element name="ACT_IND_BASE">
      <inlineType name="NVARCHAR" primitiveType="NVARCHAR" length="1" precision="0" scale="0"/>
    </element>
    <element name="INACT_FIN_TYPE">
      <inlineType primitiveType="NVARCHAR" length="3" precision="3" scale="0"/>
    </element>
    <element name="INACT_TOESL_CAT">
      <inlineType primitiveType="NVARCHAR" length="3" precision="3" scale="0"/>
    </element>
    <element name="INACT_IND_BASE">
      <inlineType name="NVARCHAR" primitiveType="NVARCHAR" length="1" precision="0" scale="0"/>
    </element>
    <element name="TOESL_CAT_A_I_FILE">
      <inlineType primitiveType="NVARCHAR" length="2" precision="2" scale="0"/>
    </element>
    <element name="RELNR_DEPOT_FILE">
      <inlineType primitiveType="NVARCHAR" length="5" precision="5" scale="0"/>
    </element>
    <element name="ACT_INACT_IND_RELNR_DEPOT">
      <inlineType name="NVARCHAR" primitiveType="NVARCHAR" length="2" precision="0" scale="0"/>
    </element>
    <element name="TMP_SALDO">
      <inlineType primitiveType="DECIMAL" length="17" precision="17" scale="3"/>
    </element>
    <element name="TMP_BOEK">
      <inlineType name="DECIMAL" primitiveType="DECIMAL" length="17" precision="0" scale="2"/>
    </element>
    <element name="SALDO_VJ" transparentFilter="false">
      <endUserTexts label="SALDO_VJ"/>
      <inlineType primitiveType="DECIMAL" length="17" precision="17" scale="2"/>
    </element>
    <input>
      <viewNode xsi:type="View:JoinNode">#/0/Join_TMP_SALDO</viewNode>
      <mapping xsi:type="Type:ElementMapping" targetName="RELNR" sourceName="RELNR"/>
      <mapping xsi:type="Type:ElementMapping" targetName="VOLGNR" sourceName="VOLGNR"/>
      <mapping xsi:type="Type:ElementMapping" targetName="VZ_CD_PA" sourceName="VZ_CD_PA"/>
      <mapping xsi:type="Type:ElementMapping" targetName="ACT_FIN_TYPE" sourceName="ACT_FIN_TYPE"/>
      <mapping xsi:type="Type:ElementMapping" targetName="ACT_TOESL_CAT" sourceName="ACT_TOESL_CAT"/>
      <mapping xsi:type="Type:ElementMapping" targetName="ACT_IND_BASE" sourceName="ACT_IND_BASE"/>
      <mapping xsi:type="Type:ElementMapping" targetName="INACT_FIN_TYPE" sourceName="INACT_FIN_TYPE"/>
      <mapping xsi:type="Type:ElementMapping" targetName="INACT_TOESL_CAT" sourceName="INACT_TOESL_CAT"/>
      <mapping xsi:type="Type:ElementMapping" targetName="INACT_IND_BASE" sourceName="INACT_IND_BASE"/>
      <mapping xsi:type="Type:ElementMapping" targetName="TOESL_CAT_A_I_FILE" sourceName="TOESL_CAT_A_I_FILE"/>
      <mapping xsi:type="Type:ElementMapping" targetName="RELNR_DEPOT_FILE" sourceName="RELNR_DEPOT_FILE"/>
      <mapping xsi:type="Type:ElementMapping" targetName="ACT_INACT_IND_RELNR_DEPOT" sourceName="ACT_INACT_IND_RELNR_DEPOT"/>
      <mapping xsi:type="Type:ElementMapping" targetName="TMP_SALDO" sourceName="TMP_SALDO"/>
      <mapping xsi:type="Type:ElementMapping" targetName="SALDO_VJ" sourceName="SALDO_VJ"/>
    </input>
    <input>
      <viewNode xsi:type="View:Aggregation">#/0/TMP_BOEK_SUM_BOEK</viewNode>
      <mapping xsi:type="Type:ElementMapping" targetName="TMP_BOEK" sourceName="BOEK_PRE"/>
    </input>
    <layout xCoordinate="609" yCoordinate="243" width="243" height="82" expanded="true"/>
    <join leftInput="#/0/Join_TMP_BOEK/Join_TMP_SALDO" rightInput="#/0/Join_TMP_BOEK/TMP_BOEK_SUM_BOEK" joinType="inner" cardinality="C1_1">
      <leftElementName>RELNR</leftElementName>
      <leftElementName>ACT_INACT_IND_RELNR_DEPOT</leftElementName>
      <rightElementName>RELNR</rightElementName>
      <rightElementName>ACT_INACT_IND_RELNR_DEPOT</rightElementName>
    </join>
  </viewNode>
  <viewNode xsi:type="View:Projection" name="TMP_YEAR_Calc">
    <endUserTexts label=" "/>
    <element name="RELNR">
      <endUserTexts label="RELNR"/>
      <inlineType primitiveType="NVARCHAR" length="5" precision="5" scale="0"/>
    </element>
    <element name="VOLGNR">
      <endUserTexts label="VOLGNR"/>
      <inlineType primitiveType="NVARCHAR" length="7" precision="7" scale="0"/>
    </element>
    <element name="VZ_CD_PA">
      <endUserTexts label="VZ_CD_PA"/>
      <inlineType primitiveType="NVARCHAR" length="4" precision="4" scale="0"/>
    </element>
    <element name="ACT_FIN_TYPE">
      <inlineType primitiveType="NVARCHAR" length="3" precision="3" scale="0"/>
    </element>
    <element name="ACT_TOESL_CAT">
      <inlineType primitiveType="NVARCHAR" length="3" precision="3" scale="0"/>
    </element>
    <element name="ACT_IND_BASE">
      <inlineType name="NVARCHAR" primitiveType="NVARCHAR" length="1" precision="0" scale="0"/>
    </element>
    <element name="INACT_FIN_TYPE">
      <inlineType primitiveType="NVARCHAR" length="3" precision="3" scale="0"/>
    </element>
    <element name="INACT_TOESL_CAT">
      <inlineType primitiveType="NVARCHAR" length="3" precision="3" scale="0"/>
    </element>
    <element name="INACT_IND_BASE">
      <inlineType name="NVARCHAR" primitiveType="NVARCHAR" length="1" precision="0" scale="0"/>
    </element>
    <element name="TOESL_CAT_A_I_FILE">
      <inlineType primitiveType="NVARCHAR" length="2" precision="2" scale="0"/>
    </element>
    <element name="RELNR_DEPOT_FILE">
      <inlineType primitiveType="NVARCHAR" length="5" precision="5" scale="0"/>
    </element>
    <element name="ACT_INACT_IND_RELNR_DEPOT">
      <inlineType name="NVARCHAR" primitiveType="NVARCHAR" length="2" precision="0" scale="0"/>
    </element>
    <element name="TMP_SALDO">
      <inlineType primitiveType="DECIMAL" length="17" precision="17" scale="3"/>
    </element>
    <element name="TMP_BOEK">
      <inlineType name="DECIMAL" primitiveType="DECIMAL" length="17" precision="0" scale="2"/>
    </element>
    <element name="SALDO_VJ" transparentFilter="false">
      <endUserTexts label="SALDO_VJ"/>
      <inlineType primitiveType="DECIMAL" length="17" precision="17" scale="2"/>
    </element>
    <element name="TMP_YEAR" aggregationBehavior="NONE">
      <inlineType name="DECIMAL" primitiveType="DECIMAL" length="17" scale="2"/>
      <calculationDefinition language="COLUMN_ENGINE">
        <formula>round(if(&quot;TMP_BOEK&quot;!= 0, (&quot;TMP_SALDO&quot; / &quot;TMP_BOEK&quot;),0),0)</formula>
      </calculationDefinition>
    </element>
    <element name="MAX_DUUR_INDEXATIE_ACT_pre" aggregationBehavior="NONE">
      <inlineType name="DECIMAL" primitiveType="DECIMAL" length="3"/>
      <calculationDefinition language="COLUMN_ENGINE">
        <formula>decfloat(if(in(&quot;ACT_INACT_IND_RELNR_DEPOT&quot;, 'A', 'AI'), round(if(&quot;TMP_BOEK&quot;!= 0, (&quot;SALDO_VJ&quot; / &quot;TMP_BOEK&quot;),0),0),0))</formula>
      </calculationDefinition>
    </element>
    <element name="MAX_DUUR_INDEXATIE_INACT_pre" aggregationBehavior="NONE">
      <inlineType name="DECIMAL" primitiveType="DECIMAL" length="3"/>
      <calculationDefinition language="COLUMN_ENGINE">
        <formula>decfloat(if(in(&quot;ACT_INACT_IND_RELNR_DEPOT&quot;, 'I', 'AI'), round(if(&quot;TMP_BOEK&quot;!= 0, (&quot;SALDO_VJ&quot; / &quot;TMP_BOEK&quot;),0),0),0))</formula>
      </calculationDefinition>
    </element>
    <element name="I_MAX_DUUR_INDEXATIE_ACT" aggregationBehavior="NONE">
      <inlineType name="NVARCHAR" primitiveType="NVARCHAR" length="3"/>
      <calculationDefinition language="COLUMN_ENGINE">
        <formula>if(&quot;MAX_DUUR_INDEXATIE_ACT_pre&quot; > 0, '', &#xD;
if(&quot;MAX_DUUR_INDEXATIE_ACT_pre&quot; &lt; 0, string(&quot;MAX_DUUR_INDEXATIE_ACT_pre&quot; * -1), &#xD;
string(&quot;MAX_DUUR_INDEXATIE_ACT_pre&quot;)))</formula>
      </calculationDefinition>
    </element>
    <element name="I_MAX_DUUR_INDEXATIE_INACT" aggregationBehavior="NONE">
      <inlineType name="NVARCHAR" primitiveType="NVARCHAR" length="3"/>
      <calculationDefinition language="COLUMN_ENGINE">
        <formula>if(&quot;MAX_DUUR_INDEXATIE_INACT_pre&quot; > 0, '', &#xD;
if(&quot;MAX_DUUR_INDEXATIE_INACT_pre&quot; &lt; 0, string(&quot;MAX_DUUR_INDEXATIE_INACT_pre&quot; * -1), &#xD;
string(&quot;MAX_DUUR_INDEXATIE_INACT_pre&quot;)))</formula>
      </calculationDefinition>
    </element>
    <element name="MAX_DUUR_INDEXATIE_ACT_future" aggregationBehavior="NONE">
      <inlineType name="NVARCHAR" primitiveType="NVARCHAR" length="3"/>
      <calculationDefinition language="COLUMN_ENGINE">
        <formula>if(in(&quot;ACT_INACT_IND_RELNR_DEPOT&quot;, 'A', 'AI'), round(if(&quot;TMP_BOEK&quot;!= 0, (&quot;TMP_SALDO&quot; / &quot;TMP_BOEK&quot;),0),0),0)</formula>
      </calculationDefinition>
    </element>
    <element name="MAX_DUUR_INDEXATIE_INACT_future" aggregationBehavior="NONE">
      <inlineType name="NVARCHAR" primitiveType="NVARCHAR" length="3"/>
      <calculationDefinition language="COLUMN_ENGINE">
        <formula>if(in(&quot;ACT_INACT_IND_RELNR_DEPOT&quot;, 'I', 'AI'), round(if(&quot;TMP_BOEK&quot;!= 0, (&quot;TMP_SALDO&quot; / &quot;TMP_BOEK&quot;),0),0),0)</formula>
      </calculationDefinition>
    </element>
    <element name="MAX_DUUR_INDEXATIE_ACT" aggregationBehavior="NONE">
      <endUserTexts label="MAX_DUUR_INDEXATIE_ACT"/>
      <inlineType name="NVARCHAR" primitiveType="NVARCHAR" length="3"/>
      <calculationDefinition language="COLUMN_ENGINE">
        <formula>rightstr(&quot;I_MAX_DUUR_INDEXATIE_ACT&quot;,3)</formula>
      </calculationDefinition>
    </element>
    <element name="MAX_DUUR_INDEXATIE_INACT" aggregationBehavior="NONE">
      <endUserTexts label="MAX_DUUR_INDEXATIE_ACT"/>
      <inlineType name="NVARCHAR" primitiveType="NVARCHAR" length="3"/>
      <calculationDefinition language="COLUMN_ENGINE">
        <formula>rightstr(&quot;I_MAX_DUUR_INDEXATIE_INACT&quot;,3)</formula>
      </calculationDefinition>
    </element>
    <input>
      <viewNode xsi:type="View:JoinNode">#/0/Join_TMP_BOEK</viewNode>
      <mapping xsi:type="Type:ElementMapping" targetName="RELNR" sourceName="RELNR"/>
      <mapping xsi:type="Type:ElementMapping" targetName="VOLGNR" sourceName="VOLGNR"/>
      <mapping xsi:type="Type:ElementMapping" targetName="VZ_CD_PA" sourceName="VZ_CD_PA"/>
      <mapping xsi:type="Type:ElementMapping" targetName="ACT_FIN_TYPE" sourceName="ACT_FIN_TYPE"/>
      <mapping xsi:type="Type:ElementMapping" targetName="ACT_TOESL_CAT" sourceName="ACT_TOESL_CAT"/>
      <mapping xsi:type="Type:ElementMapping" targetName="ACT_IND_BASE" sourceName="ACT_IND_BASE"/>
      <mapping xsi:type="Type:ElementMapping" targetName="INACT_FIN_TYPE" sourceName="INACT_FIN_TYPE"/>
      <mapping xsi:type="Type:ElementMapping" targetName="INACT_TOESL_CAT" sourceName="INACT_TOESL_CAT"/>
      <mapping xsi:type="Type:ElementMapping" targetName="INACT_IND_BASE" sourceName="INACT_IND_BASE"/>
      <mapping xsi:type="Type:ElementMapping" targetName="TOESL_CAT_A_I_FILE" sourceName="TOESL_CAT_A_I_FILE"/>
      <mapping xsi:type="Type:ElementMapping" targetName="RELNR_DEPOT_FILE" sourceName="RELNR_DEPOT_FILE"/>
      <mapping xsi:type="Type:ElementMapping" targetName="ACT_INACT_IND_RELNR_DEPOT" sourceName="ACT_INACT_IND_RELNR_DEPOT"/>
      <mapping xsi:type="Type:ElementMapping" targetName="TMP_SALDO" sourceName="TMP_SALDO"/>
      <mapping xsi:type="Type:ElementMapping" targetName="TMP_BOEK" sourceName="TMP_BOEK"/>
      <mapping xsi:type="Type:ElementMapping" targetName="SALDO_VJ" sourceName="SALDO_VJ"/>
      <layout xCoordinate="70" yCoordinate="30"/>
    </input>
    <layout xCoordinate="602" yCoordinate="165" width="256" height="64" expanded="true"/>
  </viewNode>
  <viewNode xsi:type="View:Projection" name="Projection">
    <element name="RELNR" aggregationBehavior="NONE" drillDownEnablement="DRILL_DOWN">
      <endUserTexts label="RELNR"/>
      <inlineType primitiveType="NVARCHAR" length="5" precision="5" scale="0"/>
    </element>
    <element name="VOLGNR" aggregationBehavior="NONE" drillDownEnablement="DRILL_DOWN">
      <endUserTexts label="VOLGNR"/>
      <inlineType primitiveType="NVARCHAR" length="7" precision="7" scale="0"/>
    </element>
    <element name="VZ_CD_PA" aggregationBehavior="NONE" drillDownEnablement="DRILL_DOWN">
      <endUserTexts label="VZ_CD_PA"/>
      <inlineType primitiveType="NVARCHAR" length="4" precision="4" scale="0"/>
    </element>
    <element name="MAX_DUUR_INDEXATIE_ACT" aggregationBehavior="NONE" drillDownEnablement="DRILL_DOWN">
      <endUserTexts label="MAX_DUUR_INDEXATIE_ACT"/>
      <inlineType name="NVARCHAR" primitiveType="NVARCHAR" length="3" precision="0" scale="0"/>
    </element>
    <element name="MAX_DUUR_INDEXATIE_INACT" aggregationBehavior="NONE" drillDownEnablement="DRILL_DOWN">
      <endUserTexts label="MAX_DUUR_INDEXATIE_ACT"/>
      <inlineType name="NVARCHAR" primitiveType="NVARCHAR" length="3" precision="0" scale="0"/>
    </element>
    <element name="SALDO_VJ" transparentFilter="false" aggregationBehavior="NONE" drillDownEnablement="DRILL_DOWN">
      <endUserTexts label="SALDO_VJ"/>
      <inlineType primitiveType="DECIMAL" length="17" precision="17" scale="2"/>
    </element>
    <keyElement>#/0/Projection/RELNR</keyElement>
    <keyElement>#/0/Projection/VOLGNR</keyElement>
    <keyElement>#/0/Projection/VZ_CD_PA</keyElement>
    <input>
      <viewNode xsi:type="View:Projection">#/0/TMP_YEAR_Calc</viewNode>
      <mapping xsi:type="Type:ElementMapping" targetName="RELNR" sourceName="RELNR"/>
      <mapping xsi:type="Type:ElementMapping" targetName="VOLGNR" sourceName="VOLGNR"/>
      <mapping xsi:type="Type:ElementMapping" targetName="VZ_CD_PA" sourceName="VZ_CD_PA"/>
      <mapping xsi:type="Type:ElementMapping" targetName="MAX_DUUR_INDEXATIE_ACT" sourceName="MAX_DUUR_INDEXATIE_ACT"/>
      <mapping xsi:type="Type:ElementMapping" targetName="MAX_DUUR_INDEXATIE_INACT" sourceName="MAX_DUUR_INDEXATIE_INACT"/>
      <mapping xsi:type="Type:ElementMapping" targetName="SALDO_VJ" sourceName="SALDO_VJ"/>
      <layout xCoordinate="70" yCoordinate="30"/>
    </input>
    <layout xCoordinate="660" yCoordinate="87" width="0" height="0" expanded="true"/>
  </viewNode>
  <viewLayout relativeWidthScenario="52"/>
</View:ColumnView>

""")

## Call OpenAI and show output

In [0]:
final_prompt = prompt + """ For below logic: \n""" + logic
try:
    model_output = call_model(selected_model, prompt=final_prompt)
    print(f"{model_output}")
except Exception as e:
    print(f"Error calling OpenAI API. Error message: {e}")


